# PWC - SA (Q2 2021)

## Inhoudsopgave
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li><a href="#EPM">EPM en Resourcelevel 3 (vanaf 2021 niet meer geanalyseerd)</a></li>
        <li><a href="#BW">Ophalen BW-informatie</a></li>
        <li><a href="#Controles">Enkele controles op de opgehaalde informatie</a></li>
        <li><a href="#Datakwaliteit">Datakwaliteit</a></li>
        <li><a href="#Exploratie">Data-exploratie</a></li>
        <li><a href="#Statistiek">Enkele statistieken laten zien</a></li>
        <li><a href="#Info1">Informatietoevoeging voorbereiden</a></li>
        <li><a href="#Info2">Informatietoevoeging uitvoeren</a></li>
        <li><a href="#TB">SA-informatie voor Technisch Beheer</a></li>
        <li><a href="#Indexeren">Indexeren en eventueel lineair extrapoleren</a></li>
        <li><a href="#Excel">Export naar Excel (inclusief velddefinitie)</a></li><br>
        <li><a href="#Ana1">Analyse 1: Werkelijke kosten en aantal gereedgemeld verdeeld over de boekjaren</a></li>
        <li><a href="#Ana2">Analyse 2: Controle voor dashboards</a></li>
    </ul>
</div>

<hr>

<div class="alert alert-block alert-danger">
<b>Let op:</b><br> In onderstaande cel dien je op te geven hoeveel maanden in het huidig boekjaar meegenomen zijn om uiteindelijk de cijfers van het huidig boekjaar te kunnen extrapoleren. Vul dit aantal maanden in achter het is-teken van variabele <b><code>aantal_maanden_meegenomen_in_huidig_boekjaar</code></b>.<br><br>
    
Verder dien je aan te geven wat het huidige boekjaar is. Vul het huidige boekjaar in achter het is-teken van variabele <b><code>huidig_boekjaar</code></b>.<br><br>

Ook kan je een extrapolatiemethode kiezen door het overeenkomstige getal in te vullen achter het is-teken van variabele <b><code>extrapolatiemethode</code></b>.<br><br>

Daarnaast kan je ook aangeven of je productbladnummers wilt aggregeren. Voornamelijk in het begin van 2019 zijn er verschillende productbladnummers geweest ter verdere diversificatie, echter werden deze productbladnummers niet ondersteund in het SAP BW-systeem (meer dan 6 cijfers gaf problemen). Daardoor zijn de productbladnummers aangepast geweest binnen de 6 cijfers, echter vonden de medewerkers deze productbladnummers niet herkenbaar. Uiteindelijk is dit in de eerste maanden van 2019 teruggedraaid. Door aan te geven of de tijdelijk actieve productbladnummers wel of niet gecorrigeerd dienen te worden naar hun oorspronkelijke productbladnummers kan je meer informatie gebruiken over de jaargrenzen heen. Dit aangeven kan achter het is-teken van variabele <b><code>pb_correctie_2019</code></b>.<br><br>

Als laatste kun je aangeven of de bronbestanden vanaf 2021 nog uit SAP BW komen of dat deze bronbestanden vanaf 2021 uit HANA komen. Dit is belangrijk voor de wijze waarop het ingelezen en gereedgemaakt dient te worden voor verdere verwerking en analyze. Vul in of de bronbestanden vanaf 2021 uit HANA komen of nog niet door True of False in te vullen achter het is-teken van variabele <b><code>HANA_bron</code></b>.<br><br></div>

In [ ]:
# Verander eventueel het aantal achter het =-teken. 
# Verander niets aan de naam van de variabele (aantal_maanden_meegenomen_in_huidig_boekjaar)
# 6 wil zeggen: cijfers t/m juni van het huidig boekjaar zijn meegenomen.
# 8 wil zeggen: cijfers t/m augustus van het huidig boekjaar zijn meegenomen.

aantal_maanden_meegenomen_in_huidig_boekjaar = 3


# Geef ook het huidig boekjaar aan:

huidig_boekjaar = 2021


# Geef ook aan hoe je de extrapolatie van de cijfers van het huidige jaar wilt hebben d.m.v. het getal achter het =-teken
# 0 geeft aan: Geen extrapolatie, alleen de werkelijke cijfers die geïmporteerd worden
# 1 geeft aan: Extrapolatie op basis van lineariteit ((waarde / aantal_maanden_meegenomen) * 12 maanden)
# 2 geeft aan: Extrapolatie op basis van verdeling van de verwachtingscijfers naar rato (over de productbladen)

extrapolatiemethode = 2


# Geef ook aan of je de productbladnummers wilt aggregeren (voornamelijk voor de productbladnummers van 2019)
# 0 = geen correctie
# 1 = wel correctie/aggregatie van productbladnummers
pb_correctie_2019 = 1


# Geef aan of de medewerkerinformatie geanonimiseerd dient te worden
# False = Medewerkerinformatie wordt niet geanonimiseerd
# True  = Medewerkerinformatie wordt wel geanonimiseerd
anonimiseren = False

# Geef aan of de bron vanaf 2021 de HANA-bron is:
# False = Bron vanaf 2021 is geen HANA-bron, maar een BW-bron
# True  = Bron vanaf 2021 is een HANA-bron

HANA_bron = True

In [ ]:
# Ter controle nogmaals de output van het aantal maanden meegenomen, extrapolatiemethode en productbladaggregatie.
# Onderstaande regels NIET aanpassen!
print("Aantal maanden meegenomen in huidig boekjaar:", aantal_maanden_meegenomen_in_huidig_boekjaar)
print("Het huidige boekjaar is                     :", huidig_boekjaar)
print("De gekozen extrapolatiemethode is           :", extrapolatiemethode, 
      "("+list(["Geen","Lineariteit", "Verwachtingscijfers"])[extrapolatiemethode]+")")
print("De gekozen productbladaggregatie is         :", pb_correctie_2019, 
      "("+list(["Geen correctie","Wel productbladaggregatie meenemen"])[pb_correctie_2019]+")")
print("Anonimisering van medewerkersinformatie     :", list(["Nee, niet geanonimiseerd", "Ja, wel geanonimiseerd"])[anonimiseren])
print("Vanaf het jaar 2021 is de bron een          :", list(["SAP BW-bron","HANA-bron"])[HANA_bron])

In [ ]:
# Enkele benodigde modules importeren
import numpy as np                      # O.a. nodig voor de NaN (Not-a-Number/waarde onbekend)-waarden
import pandas as pd                     # Voor alle manipulaties in het dataframe, import van Excel, etc.
pd.options.display.float_format = '{:,.2f}'.format  # In dataframes de getallen voorzien van een decimale punt en 
                                                    # duizendscheidingsteken is een komma. Zonder deze instelling
                                                    # was er wel een decimale punt, echter geen duizendscheidingsteken
                                                    # wat grote getallen moeilijker leesbaar maakt.
                                                    # Verder zorgt de instelling voor afronding op 2 decimalen in de weergave
pd.options.display.max_columns = None   # Alle kolommen weergeven ipv een goed scherm breed aan kolommen
import seaborn as sns                   # Voor datavisualisaties
import matplotlib.pyplot as plt         # Voor datavisualisatie-uitbreidingen
print("Modules zijn succesvol geïmporteerd!")

<a id="EPM"></a>
## EPM en resourcelevel 3

In de 2020-versie van deze productiviteit-SA-verwerking is gekeken naar het mogelijke gebruik van EPM en resourcelevel-3. De dekking hiervan is te laat om dit te gebruiken. Verderop in de code is een harde toewijzing van functie naar binnendienst/buitendienst gemaakt. Wil je meer weten over EPM en resourcelevel-3, kijk dan naar de code uit 2020.

<a id="BW"></a>
## Ophalen BW-informatie

Vanwege grootte is dit per jaar mogelijk. Bijgaand de BW-bladwijzer voor 2021:
https://vhandwnpwd01.hec.alliander.local:43084/irj/servlet/prt/portal/prtroot/pcd!3aportal_content!2fcom.sap.pct!2fplatform_add_ons!2fcom.sap.ip.bi!2fiViews!2fcom.sap.ip.bi.bex?BOOKMARK=050QF2GK9YJAUQC9MVJWQ017T (NB: Filter op boekjaar in de BW-query en geeft boekjaar bij variabelenscherm aan!)

Bestanden worden achtereenvolgens als SA2017.xls, SA2018.xls, SA2019.xls, SA2020.xls en SA2021.xls weggeschreven direct vanuit de webkant van SAP BW. Daarom is de .xls-file ook geen echte xls-file, maar een HTML-file.

### Inlezen data deel 1 (2017) met stap-voor-stap aanpassingen

In [ ]:
# Lees bestand in en laat eerste 5 regels zien
file = pd.read_html("SA2017.xls")  #, decimal=",")
df_sa = file[1]
display(df_sa.head())
print("De kolomkoppen dienen nog aangepast te worden. Deze staan in regel 1 en 2 (denk aan zero-indexing)")

In [ ]:
# Neem de omschrijving uit regel 2 (nul-indexed) over als kolomkop.
headerlist = df_sa.iloc[1].to_list()
# Voor kolom 22 en verder (nul-indexed), gebruik daar als kolomkop de omschrijving van regel 1
headerlist[21:] = df_sa.iloc[0].to_list()[21:]
# Maak kolomkoppen uniek door er "omschrijving" aan toe te voegen
headerlist[10] = headerlist[10]+"omschrijving"
headerlist[12] = headerlist[12]+"omschrijving"
headerlist[14] = headerlist[14]+"omschrijving"
headerlist[17] = headerlist[17]+"omschrijving"
headerlist[20] = headerlist[19]+"omschrijving"
# Plaats de kolomkoppen in het dataframe
df_sa.columns = headerlist
df_sa.head()

In [ ]:
# Verwijder de regels met index 0 en 1
df_sa.drop([df_sa.index[0] , df_sa.index[1]], inplace=True)
df_sa.head()

### Inlezen rest van de data (2018 en verder uit SAP BW) in één keer

In [ ]:
# Lees nu de overige dataframes binnen
print("Vooraf heeft Dataframe df_sa de volgende shape:", df_sa.shape)

filenamen = ["SA2018.xls","SA2019.xls","SA2020.xls"]

# Vervolgbewerking afhankelijk van HANA-bron
if not(HANA_bron): filenamen.append("SA2021.xls")                       # HANA_bron is nog niet actief. 2021 en verder toevoegen

for filenaam in filenamen:
    file = pd.read_html(filenaam) #, decimal=",")
    df_temp = file[1]                                                   # Benodigd dataframe is het tweede dataframe
    df_temp.drop([df_temp.index[0] , df_temp.index[1]], inplace=True)   # Verwijder rij 1 en 2
    df_temp.columns = headerlist                                        # Zet de kolomkoppen goed
    print(filenaam, "heeft shape:", df_temp.shape)
    df_sa = pd.concat([df_sa,df_temp])                                  # Plak de data onderaan erbij
print("Achteraf heeft Dataframe df_sa de volgende shape:", df_sa.shape)

<a id="Controles"></a>
## Enkele controles op de opgehaalde informatie vanuit SAP BW

**Controles op decimale punt**

In [ ]:
# Korte check bij serviceorder 7400008573. Deze heeft een cumulatieve werkelijke kosten van 134,97
display(df_sa[df_sa["Serviceorder"]=="7400008573"][["Serviceorder","Cumulatieve werkelijkekosten"]])
print("We missen de decimale punt")

In [ ]:
# Getalkolommen aanpassen naar decimalen
kolommen = ["Cumulatieve werkelijkekosten", 
       "Personeelskostencumulatief", 
       "Materiaalkostencumulatief", 
       "Dienstenkostencumulatief",
       "Intercompanykostencumulatief",
       "Werkelijke urenTotaal"]
df_sa[kolommen] = df_sa[kolommen].apply(lambda x: pd.to_numeric(x.astype(str)
                                                   .str.replace('.','').str.replace(',','.'), errors='coerce')/100)
display(df_sa[df_sa["Serviceorder"]=="7400008573"][["Serviceorder","Cumulatieve werkelijkekosten"]])
print("Decimale punt staat nu correct.")

In [ ]:
# Aantalkolom aanpassen van string (object) naar getal (float)
print("Datatype vooraf:",df_sa["SA: Aantalgereedgemeld(totaal)"].dtype)
df_sa["SA: Aantalgereedgemeld(totaal)"] = df_sa["SA: Aantalgereedgemeld(totaal)"].astype(float)
print("Datatype achteraf:",df_sa["SA: Aantalgereedgemeld(totaal)"].dtype)

In [ ]:
# Wat is het totaal van boekjaar 2017?
print("Totaal van boekjaar 2017",df_sa[df_sa["Boekjaar"]=="Kalenderjaar, 4 spec. perioden 2017"]["Cumulatieve werkelijkekosten"].sum())
print("Volgens SAP BW zou de uitkomst hiervan 11.734.419,43 moeten zijn.")
if df_sa[df_sa["Boekjaar"]=="Kalenderjaar, 4 spec. perioden 2017"]["Cumulatieve werkelijkekosten"].sum() == 11734419.43:
    print("Beide getallen zijn gelijk aan elkaar, dus de data klopt!")
else:
    # De onderstaande regel wordt alleen afgedrukt indien de waarde van 2017 NIET overeenkomt met het vaste getal 11.734.419,43
    print("WE HEBBEN EEN PROBLEEM !!!   DE DATA KLOPT NIET !!!")
    raise Exception('We hebben een probleem. De data klopt niet!')

**Aanpassen boekjaar**

In [ ]:
# Pas het boekjaar aan naar de laatste 4 karakters waarin het jaar staat
display(df_sa["Boekjaar"].head(3).to_frame())
print("Boekjaar aanpassen naar alleen het jaar")
df_sa["Boekjaar"] = df_sa["Boekjaar"].str[-4:]
display(df_sa[["Boekjaar"]].head(3))

<a id="HANA"></a>
## Ophalen HANA-informatie

### Inlezen data vanaf 2021 vanuit HANA-bron (nieuw formaat)

In [ ]:
# Vervolgbewerking afhankelijk van HANA-bron
if HANA_bron:
    # HANA-bron is actief
    str_xls_tabbladnaam_productblad = "SA-universe"
    str_xls_tabbladnaam_csorder     = "CS-order populatie"

    # Gebruikte filenamen voor HANA-bestanden (lijst dient jaarlijks uitgebreid te worden)
    filenamen = ["SA2021new.xlsx"]
    
    for filenaam in filenamen:
        # Lees Excelsheet in
        print("Bezig met bestandsnaam:", filenaam)
        xlsx = pd.ExcelFile(filenaam)

        # Haal Exceltabblad mbt Productbladnummer op en plaats deze in een Pandas Dataframe (df_pblad)
        df_pblad = xlsx.parse(sheet_name=str_xls_tabbladnaam_productblad,
                             dtype = {"Jaarplan Van":object, "Bedrijfsnummer":object, "Boekjaar":object, 
                                      "Opportunity nummer":object, "Productblad":object})
        print("Informatie uit SA-universe ingelezen:", df_pblad.shape)

        # Haal Exceltabblad mbt CS-orders op en plaats deze in een Pandas Dataframe (df_cs)
        df_cs = xlsx.parse(sheet_name=str_xls_tabbladnaam_csorder,
                          dtype = {"Bedrijfsnummer":object, "Boekjaar":object, "Opportunity nummer":object,
                                   "Serviceorder":object, "Medewerker":object})
        print("Informatie uit populatie ingelezen:", df_cs.shape)
        
        # Verwijder voorloopnullen van de serviceorder/cs-order (zijn ook niet aanwezig in andere dataframes)
        df_cs["Serviceorder"] = df_cs["Serviceorder"].str.lstrip("0")
        
        # Serviceorders met productblad(omschrijvingen) en routinggroep losweken uit SA-universe
        df_pbladnrs = df_pblad[["Serviceorder", "Productblad", "Productbladomschrijving", "Routinggroep"]]
        
        # Productblad(omschrijvingen) en routinggroep aan populatie toevoegen op basis van serviceorder
        df_cs = df_cs.merge(df_pbladnrs, on="Serviceorder", how="inner", validate="many_to_one")
        print("Informatie uit populatie na uitbreiding met productblad en productbladomschrijving:", df_cs.shape)

        # Jaar uit bestandnaam halen
        jaar = filenaam[2:6]
        print("Jaar uit bestandsnaam ("+filenaam+") (positie 3 t/m 6):", jaar)
        
        # Maak apart dataframe van CS-orders
        df_cskost = df_cs[df_cs["Cumulatieve werkelijkekosten"] != 0.0 ][["Serviceorder"]]

        # Filter dataframe uit SA-universe (productblad) o.b.v. jaar en CS-order
        df_pblad = df_pblad[((df_pblad["Jaarplan Van"]==jaar) | (df_pblad["Serviceorder"].isin(df_cskost["Serviceorder"])))]
        display(df_pblad.groupby("Jaarplan Van")["Jaarplan Van"].count().to_frame())
        print("Informatie uit SA-universe na filtering o.b.v. jaar en CS-order:", df_pblad.shape)

        # Voeg de gefilterde SA-universe-informatie toe aan de populatie
        df_cs = df_cs.append(df_pblad, ignore_index=True, sort=False)
        print("Informatie uit populatie na toevoeging SA-universe-informatie:", df_cs.shape)
        
        # Bekijk onnodige kolommen
        print("Kolommen die momenteel nog niet nodig zijn:\n", list(set(df_cs.columns).difference(set(df_sa.columns))))
        
        df_cs.drop(list(set(df_cs.columns).difference(set(df_sa.columns))), axis=1, inplace=True)
        print("Informatie uit populatie na verwijdering onnodige kolommen:", df_cs.shape)

        print("Groot dataframe (df_sa) voor samenvoeging met de populatie", df_sa.shape)
        df_sa = df_sa.append(df_cs, ignore_index=True, sort=False)
        print("Groot dataframe (df_sa) na samenvoeging met de populatie", df_sa.shape)
    
else:
    print("Er wordt nog geen HANA-broninformatie ingelezen")

### Anonimisering

In [ ]:
# Vervolgbewerking afhankelijk van de waarde van het veld anonimiseren. 
# Dit veld kan in het begin van deze Jupyter-notebook aangepast worden.
if anonimiseren:
    df_sa["Medewerkeromschrijving"] = "Vanwege privacy staat hier niet meer de echte naam vermeld"
    print("Anonimisering van medewerkerinformatie toegepast.")
else:
    print("Anonimisering van medewerkerinformatie niet toegepast.")

<a id="Datakwaliteit"></a>
## Datakwaliteit

In [ ]:
# Opdrachtgever TenneT is soms als Tennet geschreven
display(df_sa.groupby("RB: Opdrachtgever")["RB: Opdrachtgever"].count().to_frame())
# Vervang Tennet door TenneT
print("Tennet vervangen door TenneT")
df_sa.loc[df_sa["RB: Opdrachtgever"]=="Tennet",["RB: Opdrachtgever"]] = "TenneT"
# Controle of aanpassing geslaagd is
print("Datakwaliteit na aanpassing.")
display(df_sa.groupby("RB: Opdrachtgever")["RB: Opdrachtgever"].count().to_frame())

In [ ]:
# Verantwoordelijke werkplek is soms met voorvoegsel "8400/" en soms zonder. Programmatuur verwacht "8400/" verderop.
print(df_sa[df_sa["Verantw. werkplek"].notnull()]["Verantw. werkplek"].unique())
print("Aantal verschillende verantwoordelijke werkplekken met '8400/':", df_sa[df_sa["Verantw. werkplek"].str.startswith("8400/")]["Verantw. werkplek"].nunique())
print("Aantal verschillende verantwoordelijke werkplekken zonder '8400/':", df_sa[~df_sa["Verantw. werkplek"].str.startswith("8400/")]["Verantw. werkplek"].nunique())
print(df_sa.shape)
# Datakwaliteit aanpassen
print("\n8400/ toevoegen voor sommige regels\n")
df_sa.loc[~df_sa["Verantw. werkplek"].str.startswith("8400/"),["Verantw. werkplek"]] = "8400/" + df_sa["Verantw. werkplek"]
# Controle of aanpassing geslaagd is
print(df_sa[df_sa["Verantw. werkplek"].notnull()]["Verantw. werkplek"].unique())
print("Aantal verschillende verantwoordelijke werkplekken met '8400/':", df_sa[df_sa["Verantw. werkplek"].str.startswith("8400/")]["Verantw. werkplek"].nunique())
print("Aantal verschillende verantwoordelijke werkplekken zonder '8400/':", df_sa[~df_sa["Verantw. werkplek"].str.startswith("8400/")]["Verantw. werkplek"].nunique())
print(df_sa.shape)

In [ ]:
# Overzicht aantal unieke functies in het dataframe
print("Er zijn",df_sa["Functie"].nunique(),"unieke functies in totaal:")
display(df_sa[df_sa["Functie"].notnull()]["Functie"].unique())
print("Verwijderen van de A-, B-, C-, D- en E-verdeling geeft onderstaand resultaat:")
df_sa.loc[df_sa["Functie"].str[-2:].isin([" A", " B", " C", " D", " E"]),["Functie"]] = df_sa["Functie"].str[:-2]
print("Er zijn",df_sa["Functie"].nunique(),"unieke functies in totaal:")
display(df_sa[df_sa["Functie"].notnull()]["Functie"].unique())
print(df_sa.shape)

<a id="Exploratie"></a>
## Data-exploratie algemeen

**Cumulatieve werkelijke kosten per boekjaar bekijken**

In [ ]:
# Eenvoudige controle op de Cumulatieve werkelijke kosten per boekjaar
display(df_sa.pivot_table(index="RB: Opdrachtgever", columns="Boekjaar", values="Cumulatieve werkelijkekosten",
                          aggfunc="sum", margins=True))
#df_sa.groupby(["Boekjaar","RB: Opdrachtgever"])["Cumulatieve werkelijkekosten"].sum()\
#    .plot(kind="bar", title="Cumulatieve werkelijke kosten per boekjaar per opdrachtgever", color=["blue","red"]);
#plt.show()
plt.figure(figsize=(15,6))
sns.barplot(data=df_sa, x="Boekjaar",y="Cumulatieve werkelijkekosten", hue="RB: Opdrachtgever", palette=["dimgrey", "darkgrey"], estimator=sum, ci=None);
plt.title("Cumulatieve werkelijke kosten per boekjaar per opdrachtgever");

**Controle op het voorkomen van niet-aanwezige waarden in de SAP BW-notatie en dit omzetten naar pandas-waarden**

In [ ]:
# Hoe zit het met het voorkomen van hekjes ("#") en de tekst "Niet toegewezen", 
# beide door SAP BW gebruikt voor onbekende waarden
print("Aantal voorkomens van het in SAP BW gebruikte alternatief voor onbekende of niet-aanwezige waarden:")
df_sa.isin(['#',"Niet toegewezen"]).sum(axis=0)

In [ ]:
# Vervang het hekje ("#") en de tekst "Niet toegewezen" door de NaN-waarde (Not-a-Number) in twee stappen
df_sa = df_sa.replace('#',np.NaN)
df_sa = df_sa.replace('Niet toegewezen',np.NaN)

In [ ]:
# Hoe zit het met de niet aanwezige waarden? (NA of NaN)
print("Totaaloverzicht na aanpassing naar het NaN-formaat van alle ontbrekende of niet-aanwezige informatie:")
df_sa.isna().sum()

**Aanpassen van algemene object-datatypes naar categorie en een datatype naar getal (float)**

In [ ]:
# Enkele datatypes aanpassen naar categorie
for kolom in ['RB: Soort werk', 'Bedrijfsnummer', 'Boekjaar', 'RB: Opdrachtgever', 'RB: Business unit']:
    df_sa[kolom] = df_sa[kolom].astype('category')
# Onderstaande aanpassing naar Float is niet meer nodig. Is al eerder in de programmatuur gedaan.
#df_sa["SA: Aantalgereedgemeld(totaal)"] = df_sa["SA: Aantalgereedgemeld(totaal)"].astype('float64')

In [ ]:
# Hoe zien de datatypes er nu uit?
df_sa.info()

<a id="Statistiek"></a>
## Enkele statistieken laten zien

**Algemene statistieken**

In [ ]:
# Laat eens wat statistieken zien van het DataFrame
df_sa.describe(include="all").T

**Aantal unieke functies**

In [ ]:
# Overzicht aantal unieke functies in het dataframe
print("Er zijn",df_sa["Functie"].nunique(),"unieke functies in totaal:")
display(df_sa[df_sa["Functie"].notnull()]["Functie"].unique())
print("Er zijn",df_sa["Functie"].isna().sum(),"regels SA-informatie zonder een waarde in de kolom 'Functie'.")
print("Er zijn",df_sa[df_sa["Cumulatieve werkelijkekosten"] != 0]["Functie"].isna().sum(),
      "regels SA-informatie zonder een waarde in de kolom 'Functie' waarbij de cumulatieve werkelijke kosten ongelijk 0 is.")
print("Dit laatste is van belang om de cumulatieve werkelijke kosten verder te kunnen toewijzen.")

**Aantal unieke productbladnummers en meer inzicht in ontbrekende productbladnummers**

In [ ]:
# Overzicht aantal unieke productbladnummers in het dataframe SA-informatie, geef eerste 10 weer
print("Er zijn",df_sa[df_sa["Productblad"].notnull()]["Productblad"].nunique(),"unieke productbladnummers in totaal, waaronder:")
print(list(df_sa[df_sa["Productblad"].notnull()]["Productblad"].unique()[:10]))
print("Er zijn",df_sa["Productblad"].isna().sum(),"regels SA-informatie zonder een waarde in de kolom 'Productblad'")

In [ ]:
# Even wat meer detailinfo met betrekking tot ontbrekende productbladinformatie, 5 grootste posten weergeven
print("Top-5 van grootste cumulatieve werkelijke kosten zonder productblad")
display(df_sa[df_sa["Productblad"].isna()].nlargest(5,"Cumulatieve werkelijkekosten"))
print("Dit zijn in totaal",df_sa[df_sa["Productblad"].isna()].shape[0] , 
      "regels met een cumulatieve werkelijke kosten van {0:.2f} EUR zonder productblad".format(
          df_sa[df_sa["Productblad"].isna()]["Cumulatieve werkelijkekosten"].sum()))
print("Deze apart vermelden in de analyse")

# Eenvoudige controle op de Cumulatieve werkelijke kosten
#df_sa[df_sa["Productblad"].isna()].groupby(["Boekjaar","RB: Opdrachtgever"])["Cumulatieve werkelijkekosten"].sum()\
#    .plot(kind="bar", title="Cumulatieve werkelijke kosten zonder productblad per boekjaar per opdrachtgever", color=["blue","red"])
#plt.show()
plt.figure(figsize=(15,6))
df_plot = df_sa[df_sa["Productblad"].isna()].groupby(["Boekjaar","RB: Opdrachtgever"])["Cumulatieve werkelijkekosten"].sum().reset_index()
sns.barplot(data=df_plot, x="Boekjaar",y="Cumulatieve werkelijkekosten", hue="RB: Opdrachtgever", palette=["dimgrey", "darkgrey"], estimator=sum, ci=None);
plt.title("Cumulatieve werkelijke kosten zonder productblad per boekjaar per opdrachtgever");
plt.show()

# Overzicht
print("\nTotaaloverzicht cumulatieve werkelijke kosten zonder productblad per boekjaar per opdrachtgever :")
display(df_sa[df_sa["Productblad"].isna()].pivot_table(index="RB: Opdrachtgever", columns="Boekjaar", values="Cumulatieve werkelijkekosten", aggfunc="sum", margins=True))

In [ ]:
# Liander heeft wel een flinke piek in 2017 aan werkelijke kosten zonder productbladinformatie
# Zeg iets over het aantal regels
print("Van de",df_sa["Productblad"].isna().sum(),"regels SA-informatie zonder een waarde in de kolom 'Productblad' zijn er",
      df_sa[(df_sa["Productblad"].isna()) & (df_sa["RB: Opdrachtgever"]== "Liander") & 
            (df_sa["Boekjaar"]== "2017")]["Cumulatieve werkelijkekosten"].count(),
            "regels voor Liander in 2017. (={0:.2f}%)".format(df_sa[(df_sa["Productblad"].isna()) & 
            (df_sa["RB: Opdrachtgever"]== "Liander") & (df_sa["Boekjaar"]== "2017")]["Cumulatieve werkelijkekosten"].count()/df_sa["Productblad"].isna().sum()*100))

# Zeg iets over de cumulatieve werkelijke kosten
print("Van de cumulatieve werkelijke kosten van {0:.2f} EUR".format(
          df_sa[df_sa["Productblad"].isna()]["Cumulatieve werkelijkekosten"].sum()), 
      "van SA-informatie zonder waarde in kolom 'Productblad' is er {0:.2f} EUR toe te kennen aan Liander in het boekjaar 2017".format(
          df_sa[(df_sa["Productblad"].isna()) & (df_sa["RB: Opdrachtgever"]== "Liander") & 
            (df_sa["Boekjaar"]== "2017")]["Cumulatieve werkelijkekosten"].sum()),"(={0:.2f}%)".format(df_sa[(df_sa["Productblad"].isna()) & (df_sa["RB: Opdrachtgever"]== "Liander") & 
            (df_sa["Boekjaar"]== "2017")]["Cumulatieve werkelijkekosten"].sum()/df_sa[df_sa["Productblad"].isna()]["Cumulatieve werkelijkekosten"].sum()*100))

# Laat de tabel zien, aflopend gesorteerd
print("\nCumulatieve werkelijke kosten zonder productblad in 2017 voor Liander, aflopend gesorteerd:")
display(df_sa[(df_sa["Productblad"].isna()) & (df_sa["RB: Opdrachtgever"]== "Liander") & (df_sa["Boekjaar"]== "2017")][['Boekjaar', 'RB: Opdrachtgever',
       'Opportunity nummer', 'Omschrijving Opportunity', 'RB: Business unit',
       'Contract+ positie', 'Omschrijving Contractpos', 'Productblad',
       'Productbladomschrijving', 'Serviceorder', 'Serviceorderomschrijving',
       'Functieplaats', 'Functieplaatsomschrijving', 'Routinggroep',
       'Medewerker', 'Medewerkeromschrijving', 'Functie', 'Verantw. werkplek',
       'Verantw. werkplekomschrijving', 'Cumulatieve werkelijkekosten',
       'Personeelskostencumulatief', 'Materiaalkostencumulatief',
       'Dienstenkostencumulatief', 'Intercompanykostencumulatief',
       'Werkelijke urenTotaal']].sort_values(by=['Cumulatieve werkelijkekosten'], ascending=False))

In [ ]:
# Liander heeft wel een flinke piek in 2017 aan werkelijke kosten zonder productbladinformatie
# Tijdelijke tabel maken voor leesbaarheid code
df_sa_geen_productblad = df_sa[(df_sa["Productblad"].isna()) & (df_sa["RB: Opdrachtgever"]== "Liander") & (df_sa["Boekjaar"]== "2017")].copy()

# Geef het totaal weer en kondig de visuals aan
print("Liander heeft in 2017 de volgende cumulatieve werkelijke kosten zonder productbladnummer: {0:.2f} EUR".format(df_sa_geen_productblad["Cumulatieve werkelijkekosten"].sum()))
print("Onderstaande 7 visuals geven meer inzicht hoe deze cumulatieve werkelijke kosten zijn opgebouwd.")
print("Een niet-aanwezige waarde (nan) wordt in blauw weergegeven.")

fig, ax = plt.subplots(nrows=7, ncols=1, sharex=True, figsize=(7,50))


# Maak voor elk van de genoemde 7 categorieën een visual
for x,i in enumerate(['RB: Business unit','Omschrijving Contractpos', 'Serviceorderomschrijving',
       'Functieplaats', 'Routinggroep', 'Functie', 'Verantw. werkplekomschrijving']):
    df_sa_geen_productblad[i] = df_sa_geen_productblad[i].astype(str)    # Omzetten naar "str" neemt NaN-waarden mee
    palette_dict = dict()
    for t in df_sa_geen_productblad[i].unique():
        if t=="nan":
            palette_dict[t] = 'blue'
        else:
            palette_dict[t] = 'dimgrey'
    #df_sa_geen_productblad.groupby(i)['Cumulatieve werkelijkekosten'].sum().to_frame().plot(kind="bar")
    #sns.barplot(data=df_sa_geen_productblad, x='Cumulatieve werkelijkekosten', y=i, estimator=sum, ci=None, hue=i, palette=palette_dict)
    sns.barplot(data=df_sa_geen_productblad, x='Cumulatieve werkelijkekosten', y=i, estimator=sum, ci=None, color=i, palette=palette_dict, ax=ax[x])
    #plt.show()

# Maak voor elk van de genoemde 7 categorieën een visual
# for i in ['RB: Business unit','Omschrijving Contractpos', 'Serviceorderomschrijving',
#        'Functieplaats', 'Routinggroep', 'Functie', 'Verantw. werkplekomschrijving']:
#     df_sa_geen_productblad[i] = df_sa_geen_productblad[i].astype(str)    # Omzetten naar "str" neemt NaN-waarden mee
#     df_sa_geen_productblad.groupby(i)['Cumulatieve werkelijkekosten'].sum().to_frame().plot(kind="bar")

**Inzicht niet-aanwezige routinggroepen**

In [ ]:
print("Cumulatieve werkelijke kosten voor niet aanwezige routinggroepen, totaal:", 
      df_sa[df_sa["Routinggroep"].isna()]["Cumulatieve werkelijkekosten"].sum())
#df_sa[df_sa["Routinggroep"].isna()].groupby("Boekjaar")["Cumulatieve werkelijkekosten"].sum().to_frame()
display(df_sa[df_sa["Routinggroep"].isna()].pivot_table(index="Boekjaar", values="Cumulatieve werkelijkekosten", aggfunc=["sum","count"], margins=True))

In [ ]:
# Een flinke piek in 2017 aan werkelijke kosten zonder routinggroepinformatie
# Tijdelijke tabel maken voor leesbaarheid code
df_sa_geen_routinggroep = df_sa[(df_sa["Routinggroep"].isna()) & (df_sa["Boekjaar"]== "2017")].copy()

# Zeg iets over het aantal regels
print("Onderstaande 7 visuals geven meer inzicht hoe deze cumulatieve werkelijke kosten zijn opgebouwd in 2017 bij ontbreken routinggroep.")
print("Een niet-aanwezige waarde (nan) wordt in blauw weergegeven.")

fig, ax = plt.subplots(nrows=7, ncols=1, sharex=True, figsize=(7,50))


# Maak voor elk van de genoemde 7 categorieën een visual
for x,i in enumerate(['RB: Business unit','Omschrijving Contractpos', 'Serviceorderomschrijving',
       'Functieplaats', 'Routinggroep', 'Functie', 'Verantw. werkplekomschrijving']):
    df_sa_geen_routinggroep[i] = df_sa_geen_routinggroep[i].astype(str)    # Omzetten naar "str" neemt NaN-waarden mee
    palette_dict = dict()
    for t in df_sa_geen_routinggroep[i].unique():
        if t=="nan":
            palette_dict[t] = 'blue'
        else:
            palette_dict[t] = 'dimgrey'
    #df_sa_geen_productblad.groupby(i)['Cumulatieve werkelijkekosten'].sum().to_frame().plot(kind="bar")
    #sns.barplot(data=df_sa_geen_productblad, x='Cumulatieve werkelijkekosten', y=i, estimator=sum, ci=None, hue=i, palette=palette_dict)
    sns.barplot(data=df_sa_geen_routinggroep, x='Cumulatieve werkelijkekosten', y=i, estimator=sum, ci=None, color=i, palette=palette_dict, ax=ax[x])
    #plt.show()

# Maak voor elk van de genoemde 7 categorieën een visual
# for i in ['RB: Business unit','Omschrijving Contractpos', 'Serviceorderomschrijving',
#        'Functieplaats', 'Productbladomschrijving', 'Functie', 'Verantw. werkplekomschrijving']:
#     df_sa_geen_routinggroep[i] = df_sa_geen_routinggroep[i].astype(str)    # Omzetten naar "str" neemt NaN-waarden mee
#     df_sa_geen_routinggroep.groupby(i)['Cumulatieve werkelijkekosten'].sum().to_frame().plot(kind="bar")

**Aantal unieke medewerkers**

In [ ]:
# Overzicht aantal unieke medewerkers in het dataframe met SA-informatie, geef eerste 10 weer
print("Er zijn",df_sa[df_sa["Medewerker"].notnull()]["Medewerker"].nunique(),
      "verschillende medewerkers in de SA-informatie, waaronder als voorbeeld deze tien:")
display(df_sa[df_sa["Medewerker"].notnull()]["Medewerker"].unique()[:10])

In [ ]:
print("Werkelijke kosten voor medewerkers zonder functie in SA-informatie.\nDit betreffen", df_sa[ (df_sa['Medewerker'].notnull()) & (df_sa['Functie'].isna())]["Medewerker"].nunique(), 
      "medewerkers met een totaal van {0:.2f} EUR".format(df_sa[ (df_sa['Medewerker'].notnull()) & (df_sa['Functie'].isna())]["Cumulatieve werkelijkekosten"].sum()))
display(df_sa[ (df_sa['Medewerker'].notnull()) & (df_sa['Functie'].isna())].pivot_table(index="Medewerker", columns="Boekjaar", values="Cumulatieve werkelijkekosten", aggfunc="sum", margins=True))

<a id="Info1"></a>
## Informatietoevoeging voorbereiden

**Medewerker-IDs toewijzen aan buitendienst en binnendienst**

In [ ]:
# Onderstaande medewerkers hebben geen functies toegekend in de SA-informatie, maar hebben wel voor 384K kosten. 
# Eerst handmatig opzoeken van de functie van de medewerkers in oude informatiesheets 
# en vervolgens samen met JohantH deze medewerkers toegewezen aan Buitendienst of Binnendienst.
dict_mw={'71698':'Binnendienst', '3473':'Binnendienst', '5879':'Binnendienst', '2852':'Buitendienst', 
         '3197':'Buitendienst', '2413':'Buitendienst', '302043':'Buitendienst', '302317':'Buitendienst'}
print('Aantal hard-toegewezen medewerkers:', len(dict_mw))
for i in set(dict_mw.values()):
    print(i,":", list(dict_mw.values()).count(i))

# Vervolgbewerking afhankelijk of alle medewerkers zonder functie toegewezen zijn aan een handmatige toewijzing
if len(set(list(df_sa[ (df_sa['Medewerker'].notnull()) & (df_sa['Functie'].isna())]["Medewerker"].unique()))
       .difference(set(list(dict_mw.keys())))) == 0:
    print("\nIedereen is toegewezen.")
else:
    # Er zijn medewerkers zonder functie die nog niet toegewezen zijn.
    print("\n",set(list(df_sa[ (df_sa['Medewerker'].notnull()) & (df_sa['Functie'].isna())]["Medewerker"].unique()))
       .difference(set(list(dict_mw.keys()))), "--> nog niet toegewezen.")
    raise Exception('Nog niet iedereen is toegewezen!')

**Functies toewijzen aan buitendienst en binnendienst**

In [ ]:
# Van 30 van de onderstaande 36 functies was geen resourcelevel 3 aanwezig. Daarom maar alle 36 functies toegewezen 
# naar Buitendienst en Binnendienst. Besproken met JohantH in Q3-2020
dict_functie={'Technisch Specialist':'Buitendienst', 'Opzichter':'Buitendienst', 'Mdw business support':'Binnendienst',
              'Technicus':'Buitendienst', 'Werkcoördinator':'Binnendienst', 'Netoperator':'Buitendienst',
              'Data Manager':'Binnendienst', 'Constructeur / Tekenaar':'Binnendienst', 'Werkvoorbereider':'Binnendienst',
              'Bouwkundige':'Buitendienst', 'Engineer':'Binnendienst', 'Technisch Uitvoerder':'Buitendienst', 
              'Bouwkundig Mdw':'Binnendienst', 'Project Realisator':'Buitendienst', 'Administratief Mdw':'Binnendienst', 
              'Monteur':'Buitendienst', 'Project Manager':'Binnendienst', 'Medewerker project support':'Binnendienst',
              'Teamleider':'Binnendienst', 'Mdw Datamanagement':'Binnendienst', 'Junior IT Professional':'Binnendienst',
              'Mdw Distributie':'Buitendienst', 'Meetspecialist':'Buitendienst', 'Trainer':'Binnendienst', 
              'Manager':'Binnendienst', 'Regio Asset Specialist':'Binnendienst', 'Technisch Beheerder':'Buitendienst', 
              'KAM Adviseur':'Binnendienst', 'Consultant':'Binnendienst', 'Werkervaringsplaats':'Binnendienst',
              'Logistiek Voorman':'Buitendienst', 'Planning & Control Mdw':'Binnendienst',
              'Juridisch Mdw':'Binnendienst', 'Chemisch Analist':'Binnendienst', 
              'Planner':'Binnendienst', 'Tactisch planner':'Binnendienst', 'Informatiemanager':'Binnendienst'}
print('Aantal hard-toegewezen functies:', len(dict_functie))
for i in set(dict_functie.values()):
    print(i,":", list(dict_functie.values()).count(i))

# Vervolgbewerking afhankelijk of alle medewerkers zonder functie toegewezen zijn aan een handmatige toewijzing
if len(set(list(df_sa[ (df_sa['Functie'].notnull())]["Functie"].unique()))
       .difference(set(list(dict_functie.keys())))) == 0:
    print("\nIedereen is toegewezen.")
else:
    # Er zijn medewerkers zonder functie die nog niet toegewezen zijn.
    print("\n",set(list(df_sa[ (df_sa['Functie'].notnull())]["Functie"].unique()))
       .difference(set(list(dict_functie.keys()))), "--> nog niet toegewezen.")
    raise Exception('Nog niet iedereen is toegewezen!')

**Routinggroepen toewijzen aan ondersteuningsorder of directe order**

In [ ]:
# Maak een lijst van alle unieke routinggroepen
routinggroep = df_sa[df_sa["Routinggroep"].notnull()]["Routinggroep"].unique()
print("Aantal routinggroepen:", len(routinggroep))

# Plaats alle routinggroepen met de letters 'OND' in de code in een dictionary met als uitkomst: "Ondersteuningsorder"
# De rest is Directe order
dict_routinggroep={}
for i in routinggroep:
    if 'OND' in i:
        dict_routinggroep[i]="Ondersteuningsorder"
    else:
        if i == "Indirect":
            dict_routinggroep[i]="Ondersteuningsorder"
        else:
            dict_routinggroep[i]="Directe order"
print("Aantal routinggroepen toegewezen:", len(dict_routinggroep), ", namelijk:", list(dict_routinggroep.keys()))
for i in set(dict_routinggroep.values()):
    print(i,":", list(dict_routinggroep.values()).count(i))

**Verantwoordelijke werkplekken toewijzen aan regio of landelijke unit (indien mogelijk)**

In [ ]:
# Uit de verantwoordelijke werkplekken kan soms een regio afgeleid worden
print("Aantal unieke verantwoordelijke werkplekken uit de geïmporteerde dataset:",df_sa[df_sa["Verantw. werkplek"].notnull()]["Verantw. werkplek"].nunique())
print(df_sa[df_sa["Verantw. werkplek"].notnull()]["Verantw. werkplek"].unique())

# Verantwoordelijke werkplek naar Regio/Landelijke unit afleiden, soms ook splitsen van waarden, etc.
# Opmaak van de dictionary zodat dit verwerkt kan worden
dict_wp={"8400/ABTB": ("(AB) Technisch Beheer", "", "1", "Uitsluiting op basis van verantwoordelijke werkplek"), #Projectmanagers -> Correctie
         "8400/GGINOORD": ("LU T&G GGI Noord", "T&G (LU)", "1"),
         "8400/GGIOOST": ("LU T&G GGI Oost", "T&G (LU)", "1"),
         "8400/GGIWEST": ("LU T&G GGI West", "T&G (LU)", "1"),
         "8400/OTDAG": ("OTD Regio AG", "Amsterdam Gooi", "1"),
         "8400/OTDBA": ("OTD Regio AB", "Achterhoek Betuwe", "1"),
         "8400/OTDFR": ("OTD Regio FNOP", "Friesland NOP", "1"),
         "8400/OTDNH": ("OTD Regio NHN", "Noord-Holland Noord", "1"),
         "8400/OTDRL": ("OTD Regio RL", "Rijnland", "1"),
         "8400/OTDVF": ("OTD Regio VF", "Veluwe Flevo", "1"),
         "8400/STAG": ("HST Regio AG", "Amsterdam Gooi", "1"),
         "8400/STBA": ("HST Regio AB", "Achterhoek Betuwe", "1"),
         "8400/STFR": ("HST Regio FNOP", "Friesland NOP", "1"),
         "8400/STIR": ("(ST) Instandhoudingmanagement Regie", "", "1", "Uitsluiting op basis van verantwoordelijke werkplek"), #Instandhoudingmanagement -> Correctie
         "8400/STNH": ("HST Regio NHN", "Noord-Holland Noord", "1"),
         "8400/STRL": ("HST Regio RL", "Rijnland", "1"),
         "8400/STSN": ("(ST) SI Noord", "Friesland NOP", "1"),
         "8400/STSO": ("(ST) SI Oost", "Achterhoek Betuwe / Veluwe Flevo", (("Achterhoek Betuwe","0.5"),("Veluwe Flevo", "0.5")) ),
         "8400/STSS": ("LU Vermogensschakelaars", "Vermogensschakelaars (LU)", "1"),
         "8400/STST": ("LU Transformatoren", "Transformatoren (LU)", "1"),
         "8400/STSW": ("(ST) SI West", "Amsterdam Gooi / Noord-Holland Noord / Rijnland", (("Amsterdam Gooi","0.34"),("Noord-Holland Noord", "0.33"),("Rijnland", "0.33")) ),
         "8400/STTF": ("(ST) GGI Oost/FRL (Frank Jacobs)", "T&G (LU)", "1"),
         "8400/STTG": ("T&G AG (Bjorn Vocking)", "T&G (LU)", "1"),
         "8400/STTL": ("T&G RNL & KEN (Johan Leek)", "T&G (LU)", "1"),
         "8400/STTN": ("T&G FRL (René van der Heide)", "T&G (LU)", "1"),
         "8400/STTO": ("(ST) Oost(Nurettin Baydemir)", "T&G (LU)", "1"),
         "8400/STTR": ("(ST) T&G realisatie", "T&G (LU)", "1"),
         "8400/STTW": ("T&G GGI WEST (Jack Bakker)", "T&G (LU)", "1"),
         "8400/STVF": ("HST Regio VF", "Veluwe Flevo", "1"),
         "8400/T&GAG": ("LU T&G AG", "T&G (LU)", "1"),
         "8400/T&GASB": ("LU T&G Asbest", "T&G (LU)", "1"),
         "8400/T&GBA": ("LU T&G AB", "T&G (LU)", "1"),
         "8400/T&GFRL": ("LU T&G FNOP", "T&G (LU)", "1"),
         "8400/T&GNH": ("LU T&G NHN", "T&G (LU)", "1"),
         "8400/T&GRNL": ("LU T&G RL", "T&G (LU)", "1"),
         "8400/T&GVF": ("LU T&G VF", "T&G (LU)", "1"),
         "8400/VBKV": ("(VB) Kwaliteit & Veiligheid", "Kabels (LU) / Lijnen (LU)", (("Kabels (LU)","0.35"),("Lijnen (LU)", "0.65"))),
         "8400/VKRKAB": ("LU Kabels", "Kabels (LU)", "1"),
         "8400/VKRLIJN": ("LU Lijnen", "Lijnen (LU)", "1")}

print('\nAantal hard-toegewezen verantwoordelijke werkplekken middels vertaaltabel:', len(dict_wp))
templist = [x[1] for x in list(dict_wp.values())]
for i in sorted(set(templist)):
    print(i,":", templist.count(i))

assert len(dict_wp) == df_sa[df_sa["Verantw. werkplek"].notnull()]["Verantw. werkplek"].nunique(), "Aantal Verantwoordelijke werkplekken niet gelijk"

**Nog te splitsen over de regio's en landelijke units**

In [ ]:
# Maak een lijst van de verantwoordelijke werkplekken die nog gesplitst moeten gaan worden
wpsplitlist = []
for k, v in dict_wp.items():
#    print(k,v)
    if type(v[2]) != type("1"):
        wpsplitlist.append(k)
print("De data van",len(wpsplitlist),"verantwoordelijke werkplekken moeten nog gesplitst worden over de regio's of landelijke units:\n", wpsplitlist)
display(df_sa[(df_sa["Verantw. werkplek"].isin(wpsplitlist))].pivot_table(index=["RB: Opdrachtgever", "Verantw. werkplek"], 
                columns="Boekjaar", values="Cumulatieve werkelijkekosten", aggfunc=["sum", "count"], margins=True))

**Nog uit te sluiten (correctie) op basis van verantwoordelijke werkplek**

In [ ]:
# Maak een lijst van de verantwoordelijke werkplekken die nog gecorrigeerd moeten worden (uitsluiting)
wpcorrectielist = []
for k, v in dict_wp.items():
#    print(k,v)
    if len(v) > 3:
        wpcorrectielist.append(k)
print("De data van",len(wpcorrectielist),"verantwoordelijke werkplekken moeten nog uitgesloten worden:\n", wpcorrectielist)
display(df_sa[(df_sa["Verantw. werkplek"].isin(wpcorrectielist))].pivot_table(index=["RB: Opdrachtgever", "Verantw. werkplek"], 
                columns="Boekjaar", values="Cumulatieve werkelijkekosten", aggfunc=["sum", "count"], margins=True))

In [ ]:
# Vervolgbewerking afhankelijk of de productbladcorrectie/productbladaggregatie over voornamelijk 2019 wordt meegenomen
if pb_correctie_2019 == 1:
    print("Productbladcorrectie (voornamelijk gerelateerd aan het boekjaar 2019) wel meenemen.")
    # In 2019 zijn er tijdelijk een aantal productbladnummers geweest ter uitsplitsing van een oud, bestaand
    # productbladnummer. Hieronder is de vertaaltabel van in 2019 gebruikte productbladen naar oude, bestaande productbladnummers
    pb2019_dict = { '443400':'443201', '443401':'443201', '443402':'443201', '443403':'443201', '443404':'443201', 
                '443405':'443201', '443406':'443201', '443407':'443201', '443408':'443201', '443409':'443201',
                '443410':'443201', '443411':'443201', '443420':'443202', '443421':'443202', '443422':'443202',
                '443423':'443202', '443424':'443202', '443425':'443202', '443426':'443202', '443427':'443202',
                '443428':'443202', '443429':'443202', '443430':'443202', '443431':'443202', '443440':'443203',
                '443441':'443203', '443442':'443203', '443443':'443203', '443444':'443203', '443445':'443203', 
                '143900':'143026', '143901':'143026', '143905':'143032', '143906':'143032', '143910':'143211', 
                '143911':'143211', '143915':'143212', '143916':'143212', '143917':'143212', '143918':'143212', 
                '143920':'143215', '143921':'143215', '143922':'143215', '143923':'143215', '143924':'143215', 
                '143930':'143216', '143931':'143216', '143935':'143301', '143936':'143301', '143940':'143304', 
                '143941':'143304', '143942':'143304', '143945':'143325', '143946':'143325', '143947':'143325', 
                '143950':'143326', '143951':'143326', '143952':'143326', '143955':'143327', '143956':'143327', 
                '143957':'143327', '143960':'143328', '143961':'143328', '143962':'143328', '143965':'143329', 
                '143966':'143329', '143967':'143329', '143970':'143330', '143971':'143330', '143972':'143330', 
                '143975':'143331', '143976':'143331', '143977':'143331', '143980':'143332', '143981':'143332', 
                '143982':'143332', '143985':'143403', '143986':'143403'}

    # Maak een dictionary voor de bijbehorende omschrijvingen
    pbtxt_dict = dict(zip(df_sa['Productblad'], df_sa['Productbladomschrijving']))

    print("De vertaaltabel van productbladen bevat",len(pb2019_dict),"productbladnummers.")
else:
    print("Productbladcorrectie (voornamelijk gerelateerd aan het boekjaar 2019) niet meenemen.")
    pb2019_dict = dict()
    pbtxt_dict = dict()

<a id="Info2"></a>
## Informatietoevoeging uitvoeren

**Vertaal de tijdelijk gebruikte productbladnummers uit 2019 naar oude, bestaande productbladnummers**

In [ ]:
# Vertaal de tijdelijk gebruikte productbladnummers uit 2019 en pas ook de omschrijving hiervan aan.
def vertaalproductblad(row):
    """Op basis het productbladnummer wordt deze omgezet naar een andere, wanneer het productbladnummer voorkomt uit de
       dictionary met 2019-productbladnummers. Ook bijgehorende omschrijvingen worden aangepast."""
    # Vervolgbewerking afhankelijk van het voorkomen van het productbladnummer in de vertaaldictionary pb2019_dict
    if row["Productblad"] in pb2019_dict:
        row["Productblad"] = pb2019_dict[row["Productblad"]]
        # ook omschrijving aanpassen
        if row["Productblad"] in pbtxt_dict:
            row["Productbladomschrijving"] = pbtxt_dict[row["Productblad"]]
    return row

# Voer bovengenoemde functie uit voor elke rij in het dataframe
df_sa = df_sa.apply(func=vertaalproductblad, axis=1)

**Maak nieuwe kolommen aan voor toevoeging van informatie (herkenbaar aan voorvoegstel TV)**

In [ ]:
# Maak nieuwe kolommen aan (TV staat voor toevoeging) en pas datatypes aan naar object
for kolom in ['TV_medewerker', 'TV_functie', 'TV_routinggroep', 'TV_datakwaliteit', 'TV_werk', "TV_regio", "TV_correctie"]:
    df_sa[kolom] = np.nan
    df_sa[kolom] = df_sa[kolom].astype('object')
print("Nieuwe kolommen toegevoegd met lege waarde")

**Medewerker-, functie- en routinggroepinformatie toevoegen richting uitvoering of voorbereiding/nazorg. Ook correcties verwerken op basis van verantwoordelijke werkplek**

In [ ]:
# Maak een functie voor aanpassing van het dataframe. Classificeer werk obv medewerker, functie en routinggroep.
# Markeer correctie obv verantwoordelijke werkplek.
def classificeer_werk_en_correctie(row):
    """Op basis van medewerker, functie of routinggroep de informatie
       uitbreiden in dit dataframe"""
    mw = ""
    # Vervolgbewerking afhankelijk van het voorkomen van de hard-toegewezen medewerkers
    if row["Medewerker"] in dict_mw:
        row["TV_medewerker"] = dict_mw[row["Medewerker"]]
        mw = row["TV_medewerker"]
    # Vervolgbewerking afhankelijk van het voorkomen van de hard-toegewezen functies
    if row["Functie"] in dict_functie:
        row["TV_functie"] = dict_functie[row["Functie"]]
        if mw == "":
            mw = row["TV_functie"]
        elif mw != row["TV_functie"]:
            row["TV_datakwaliteit"] = "Niet OK"
        else:
            pass   # Niets aan de hand (mw == functie)
    # Vervolgbewerking afhankelijk van het voorkomen van de "indirecte" routinggroepen
    if row["Routinggroep"] in dict_routinggroep:
        row["TV_routinggroep"] = dict_routinggroep[row["Routinggroep"]]
        # Vervolgbewerking afhankelijk van routinggroep en medewerker binnen/buiten
        if (mw == "Buitendienst") and (row["TV_routinggroep"] == "Directe order"):
            # Werk = uitvoering
            row["TV_werk"] = "Uitvoering"
        else:
            # Werk = voorbereiding/nazorg
            row["TV_werk"] = "Voorbereiding of nazorg"
    
    # Vervolgbewerking afhankelijk van het voorkomen van de verantwoordelijke werkplek
    if row['Verantw. werkplek'] in dict_wp:
        # Vervolgbewerking afhankelijk van het aanwezig zijn van de correctieinformatie (alleen bij lengte 4 aanwezig)
        if len(dict_wp[row['Verantw. werkplek']]) == 4:
            row["TV_correctie"] = dict_wp[row['Verantw. werkplek']][3]  #Zero-index!!
    
#    We kijken niet meer of in de omschrijving van een serviceorder het woordje 'ondersteuning' voor komt.
#    if "ondersteuning" in row["Serviceorderomschrijving"].lower():
#        row["DI_serviceordertype"] = "Indirect"
#        lst.append(row["DI_serviceordertype"])
    return row        

In [ ]:
# Voer bovengenoemde functie uit voor elke rij in het dataframe
df_sa = df_sa.apply(func=classificeer_werk_en_correctie, axis=1)

In [ ]:
# Snel overzicht op aantal non-null-waarden (omgekeerde van NaN-waarden (Niet-aanwezige waarden))
df_sa.info()

In [ ]:
print("Aantal datakwaliteitsproblemen tussen medewerker en functie:", df_sa["TV_datakwaliteit"].notnull().sum())
# Indien er wel datakwaliteitsproblemen zijn, laat dan de eerste 5 ervan zien.
if df_sa["TV_datakwaliteit"].notnull().sum() !=0:
    display(df_sa[df_sa["TV_datakwaliteit"].notnull()].head())

**Splits de informatie op basis van verantwoordelijke werkplek over de regio's of landelijke units**

In [ ]:
# Maak kopie van te splitsen regels
# Werkset voor de nog te splitsen regels over de regio's
df_sa_werkset = df_sa[(df_sa["Verantw. werkplek"].isin(wpsplitlist))].copy()
# Regio voor de niet te splitsen over de regio's
df_sa_regio = df_sa[(~df_sa["Verantw. werkplek"].isin(wpsplitlist))].copy()

print("Origineel dataframe    :", df_sa.shape)
print("Werkset nog te splitsen:", df_sa_werkset.shape)
print("Dataframe niet splitsen:", df_sa_regio.shape)
print("Aantallen controle nog te splitsen + niet splitsen", df_sa_werkset.shape[0]+df_sa_regio.shape[0])
if df_sa_werkset.shape[0]+df_sa_regio.shape[0] == df_sa.shape[0]:
    # Dit wordt alleen maar afgedrukt wanneer er geen verschil is
    print("Dit is gelijk aan het originele dataframe")
else:
    # Dit wordt alleen maar afgedrukt wanneer er wel een verschil is
    print("ER IS EEN VERSCHIL MET HET ORIGINELE DATAFRAME VAN",df_sa_werkset.shape[0]+df_sa_regio.shape[0]-df_sa.shape[0],
          "REGELS !!")
    raise Exception('Er is een verschil met het originele dataframe!')

In [ ]:
# Maak een functie voor het opsplitsen naar regio of landelijke unit
def splits_naar_regio_of_landelijke_unit_2(row, werkplek, regio, kolommen):
    """Splits een regel op door lijsten (lists) in cellen te zetten en daarna (buiten deze functie)
       het dataframe laten 'exploden'"""

    debug = False
    if debug == True: print("Overzicht van de parameters: werkplek", werkplek, ", regio", regio, ", kolommen", kolommen)
    
    # Haal verdeelsleutel (tupel) op uit de dictionary van verantwoordelijke werkplekken
    verdeelsleutel = dict_wp[row[werkplek]][2]
    # Vervolgbewerking afhankelijk of de verdeelsleutel een tupel is
    if type(verdeelsleutel) == type( (1,2) ):
        # Ja, verdeelsleutel is een tupel
        if debug == True: print("Verdeelsleutel:", verdeelsleutel)
        dict_mainrow = dict()        # Deze dictionary wordt gebruikt voor de waardevelden met opbouw [restant, totaal]
        teller = 0                   # Geeft de positie binnen de verdeelsleutel aan
        
        # Waardevelden veiligstellen in dictionary
        for kolom in kolommen:
            dict_mainrow[kolom] = [ row[kolom], row[kolom] ]     # [ restant, totaal ] is nu nog gelijk
            row[kolom] = list()                                  # Veld in de rij wordt van het type "list"
        row[regio] = list()                                      # Regioveld wordt ook van het type "list"
        if debug == True: print("dict_mainrow:",dict_mainrow)
        
        for regioomschrijving, factor in verdeelsleutel:
            
            teller += 1                                          # Verhoog de teller met 1
            
            if debug == True: print("Teller:", teller, ", regioomschrijving:", regioomschrijving, ", factor:", factor)
            tempregio = row[regio]
            tempregio.append(regioomschrijving)
            row[regio] = tempregio
            if debug == True: print("row[regio]:", row[regio])
            
            # Behandel alle kolommen
            for kolom in kolommen:
                # Vervolgbewerking afhankelijk van de teller
                if teller == len(verdeelsleutel):
                    # We zijn in de laatste ronde
                    tempwaarde          = row[kolom]
                    tempmainwaarde      = dict_mainrow[kolom]
                    tempwaarde.append(tempmainwaarde[0])
                    row[kolom]          = tempwaarde
                else:
                    # We zijn nog niet in de laatste ronde
                    tempwaarde          = row[kolom]
                    tempmainwaarde      = dict_mainrow[kolom]
                    tempwaarde.append(round(tempmainwaarde[1]*float(factor),2))
                    tempmainwaarde[0]   = tempmainwaarde[0] - tempwaarde[-1]
                    row[kolom]          = tempwaarde
                    dict_mainrow[kolom] = tempmainwaarde
                if debug == True: print(kolom, dict_mainrow[kolom], row[kolom])
    return row

In [ ]:
# Voer bovengenoemde functie uit voor elke rij in het dataframe

# Maak van veld TV_regio een object (en geen getal/float)
df_sa_werkset["TV_regio"] = df_sa_werkset["TV_regio"].astype('object')

# Lijst van kolommen die met hun waarde gesplitst moeten worden
kolommen2 = ['Cumulatieve werkelijkekosten', 'Personeelskostencumulatief', 'Materiaalkostencumulatief',
             'Dienstenkostencumulatief', 'Intercompanykostencumulatief', 'Werkelijke urenTotaal',
             'SA: Aantalgereedgemeld(totaal)']

# Lijst van kolommen die nodig zijn voor de .explode-methode om ze te splitsen in meerdere rijen
kolommen = kolommen2[:]       # Zo maak je een echte kopie, en niet een verwijzing naar de zelfde lijst
kolommen.append("TV_regio")   # Ook het veld TV_regio dient gesplitst te worden bij de .explode()-methode

# Voorafcontrole
print("Shape van het dataframe vooraf:", df_sa_werkset.shape)
for kolom in kolommen2:
    print("Totaal van df_sa_werkset", kolom, ": {0:,.2f}".format(df_sa_werkset[kolom].sum()))

print("\nShape van het te splitsen dataframe:", df_sa_werkset.shape)

# LET OP: Zonder .reset_index(drop=True) kreeg ik met deze dataset een "ValueError: cannot reindex from a duplicate axis"
# Splits het dataframe (dataframe df_temp krijgt al toekenning van het veld "TV_regio")
df_sa_gesplitst = (df_sa_werkset.reset_index(drop=True)
           .apply(func=splits_naar_regio_of_landelijke_unit_2, axis=1, args=("Verantw. werkplek", "TV_regio", kolommen2))
           .apply(lambda x: x.explode() if x.name in kolommen else x)
           .copy() )
print("Shape van het gesplitste dataframe:", df_sa_gesplitst.shape)

print("\nShape van het dataframe achteraf:",df_sa_gesplitst.shape)
for kolom in kolommen2:
    print("Totaal van df_temp", kolom, ": {0:,.2f}".format(df_sa_gesplitst[kolom].sum()))

# Controle op Cumulatieve werkelijke kosten op 2 decimalen
if round(df_sa_werkset["Cumulatieve werkelijkekosten"].sum(),2) != \
   round(df_sa_gesplitst["Cumulatieve werkelijkekosten"].sum(),2):
    # Dit wordt alleen maar afgedrukt wanneer er wel een verschil is
    raise Exception('Er is een verschil met de bedragen tussen het dataframe vooraf en het gesplitste dataframe achteraf!')
    
print("\nAantal regels wel gevuld met regio/LU :",df_sa_gesplitst["TV_regio"].notnull().sum())
print("Aantal regels niet gevuld met regio/LU:",df_sa_gesplitst["TV_regio"].isna().sum())

# Controle op aantal lege regels mbt regio/LU
if df_sa_gesplitst["TV_regio"].isna().sum() != 0:
    # Dit wordt alleen maar afgedrukt wanneer er wel een verschil is
    raise Exception('Er zijn regels waarvan de regio/LU niet gevuld zijn!')

**Ken regio of landelijke unit toe aan niet-gesplitste dataset**

In [ ]:
# Maak een functie aan voor het toekennen aan regio of landelijke unit zonder splitsen
def regio_of_landelijke_unit(row):
    # Vervolgbewerking afhankelijk van het voorkomen van de verantwoordelijke werkplek
    if row['Verantw. werkplek'] in dict_wp:
        row["TV_regio"] = dict_wp[row['Verantw. werkplek']][1]  #Zero-index!!
    return row

# Voor de functie uit voor dit dataframe
df_sa_regio=df_sa_regio.apply(func=regio_of_landelijke_unit, axis=1)

print("Aantal regels wel gevuld met regio/LU :",df_sa_regio["TV_regio"].notnull().sum())
print("Aantal regels niet gevuld met regio/LU:",df_sa_regio["TV_regio"].isna().sum())

**Voeg gesplitste regels toe aan Dataframe met regio-/landelijke unit-toekenning**

In [ ]:
# Informatie vooraf
regio_regels_voor_samenvoeging = df_sa_regio.shape[0]
regio_cumulatieve_werkelijke_kosten_voor_samenvoeging = df_sa_regio["Cumulatieve werkelijkekosten"].sum()
print("Dataframe niet gesplitst met regio :", df_sa_regio.shape,
      "met cumulatieve werkelijke kosten {0:.2f} EUR".format(regio_cumulatieve_werkelijke_kosten_voor_samenvoeging))
print("Dataframe reeds gesplitst met regio:", df_sa_gesplitst.shape,
      "met cumulatieve werkelijke kosten {0:.2f} EUR".format(df_sa_gesplitst["Cumulatieve werkelijkekosten"].sum()))

# Het daadwerkelijk samenvoegen
df_sa_regio = df_sa_regio.append(df_sa_gesplitst,ignore_index=True)

# Controle achteraf op regels
print("Aantal regels na samenvoegen in één dataframe", df_sa_regio.shape[0])
if regio_regels_voor_samenvoeging+df_sa_gesplitst.shape[0] == df_sa_regio.shape[0]:
    # Dit wordt alleen maar afgedrukt wanneer er geen verschil is
    print("Dit is gelijk aan de som van beide dataframes voor de samenvoeging")
else:
    # Dit wordt alleen maar afgedrukt wanneer er wel een verschil is
    print("ER IS EEN VERSCHIL MET HET AANTAL REGELS VOOR EN NA SAMENVOEGING VAN ",regio_regels_voor_samenvoeging+df_sa_gesplitst.shape[0]-df_sa_regio.shape[0],
          "REGELS !!")
    raise Exception('Er is een verschil met het aantal regels tussen het dataframe voor samenvoeging en na samenvoeging!')

# Controle achteraf op cumulatieve werkelijke kosten
print("Cumulatieve werkelijke kosten na samenvoegen in één dataframe {0:.2f} EUR".format(df_sa_regio["Cumulatieve werkelijkekosten"].sum()))
# Controle op Cumulatieve werkelijke kosten op 2 decimalen
if round(regio_cumulatieve_werkelijke_kosten_voor_samenvoeging + df_sa_gesplitst["Cumulatieve werkelijkekosten"].sum(),2) != \
   round(df_sa_regio["Cumulatieve werkelijkekosten"].sum(),2):
    # Dit wordt alleen maar afgedrukt wanneer er wel een verschil is
    raise Exception('Er is een verschil met de bedragen tussen het dataframe voor samenvoeging en na samenvoeging!')
else:
    print("Er is geen verschil met de bedragen tussen het dataframe voor samenvoeging en na samenvoeging!")

**Correctie voor ontbreken productbladnummer**

In [ ]:
# Tijdelijke tabel maken voor leesbaarheid code en conversie van NaN-waarden naar "string"
df_sa_geen_productblad = df_sa_regio[(df_sa_regio["Productblad"].isna())].copy()
df_sa_geen_productblad["TV_correctie"] = df_sa_geen_productblad["TV_correctie"].astype(str)    # Omzetten naar "str" neemt NaN-waarden mee
print(len(df_sa_geen_productblad), "regels hebben geen productbladnummer.",
      "\nEen aantal van deze regels wordt al uitgesloten vanwege reeds toegekende correctie:")
display(df_sa_geen_productblad.pivot_table(index=["TV_correctie","RB: Opdrachtgever"], columns="Boekjaar", values="Cumulatieve werkelijkekosten", aggfunc=["sum","count"], margins=True))

In [ ]:
# Deze correctie wordt toegevoegd in de code om ook de correctie bij een ontbrekende routinggroep toe te voegen
Stop?

**Correctie voor ontbreken routinggroep**

In [ ]:
# Tijdelijke tabel maken voor leesbaarheid code en conversie van NaN-waarden naar "string"
df_sa_geen_routinggroep = df_sa_regio[(df_sa_regio["Routinggroep"].isna())].copy()
df_sa_geen_routinggroep["TV_correctie"] = df_sa_geen_routinggroep["TV_correctie"].astype(str)    # Omzetten naar "str" neemt NaN-waarden mee
print(len(df_sa_geen_routinggroep), "regels hebben geen routinggroep.",
      "\nEen aantal van deze regels wordt al uitgesloten vanwege reeds toegekende correctie:")
display(df_sa_geen_routinggroep.pivot_table(index=["TV_correctie","RB: Opdrachtgever"], columns="Boekjaar", values="Cumulatieve werkelijkekosten", aggfunc=["sum","count"], margins=True))

print("Een deel hiervan mist naast de routinggroep ook een productbladnummer:")
df_sa_geen_routinggroep["Productblad"] = df_sa_geen_routinggroep["Productblad"].astype(str)    # Omzetten naar "str" neemt NaN-waarden mee
display(df_sa_geen_routinggroep.pivot_table(index=["TV_correctie","Productblad","RB: Opdrachtgever"], columns="Boekjaar", values="Cumulatieve werkelijkekosten", aggfunc=["sum","count"], margins=True))

In [ ]:
# Functie om correctie voor ontbrekende productbladnummer en routinggroep te verwerken
def correctie_productblad_routinggroep(row):
    if str(row['Productblad']) == str(np.nan):
        if str(row['TV_correctie']) != str(np.nan):
            row['TV_correctie'] = row['TV_correctie'] + " | Uitsluiting op basis van ontbreken productblad"
        else:
            row['TV_correctie'] = "Uitsluiting op basis van ontbreken productblad"
    if str(row['Routinggroep']) == str(np.nan):
        if str(row['TV_correctie']) != str(np.nan):
            row['TV_correctie'] = row['TV_correctie'] + " | Uitsluiting op basis van ontbreken routinggroep"
        else:
            row['TV_correctie'] = "Uitsluiting op basis van ontbreken routinggroep"
    return row

# Voer de functie uit voor het dataframe
df_sa_regio = df_sa_regio.apply(func=correctie_productblad_routinggroep, axis=1)

In [ ]:
# Controle van de verwerking van de correcties
# Tijdelijke tabel maken voor leesbaarheid code en conversie van NaN-waarden naar "string"
df_sa_geen_routinggroep = df_sa_regio[(df_sa_regio["Routinggroep"].isna())].copy()
df_sa_geen_routinggroep["TV_correctie"] = df_sa_geen_routinggroep["TV_correctie"].astype(str)    # Omzetten naar "str" neemt NaN-waarden mee
print(len(df_sa_geen_routinggroep), "regels hebben geen routinggroep.")
display(df_sa_geen_routinggroep.pivot_table(index=["TV_correctie","RB: Opdrachtgever"], columns="Boekjaar", values="Cumulatieve werkelijkekosten", aggfunc=["sum","count"], margins=True))

<a id="TB"></a>
## SA-informatie voor Technisch Beheer

TheoB:Het veld SA: Aantal in opdracht (na huidige periode) wordt bepaald op basis van de startdatum van de order wordt het aantal toegekend aan een maand en die meetwaarde toont dan alles dat na de opgegeven huidige periode valt.

Theoretisch kan het dus voorkomen dat er orders zijn, die niet gereed zijn of niet in overloop/vervallen staan, die een startdatum van de order hebben die ligt voor de huidige periode.

Daarom is de berekening vanuit de formule SA: Aantal in opdracht totaal -/- gereed gemeld -/- overloop/vervallen een betrouwbaarder meetwaarde.

<div class="alert alert-block alert-info">
Aanvullende informatie geeft aan dat de verantwoordelijke werkplek 8400/ABTB alleen maar wordt gebruikt voor technisch beheer. De boekingsregels die gerelateerd zijn aan deze werkplek worden in de vertaling van de verantwoordelijke werkplekken naar regio's ook gemarkeerd dat ze als correctie gezien moeten worden.<br>

Technisch binnen SAP wordt er op contract+positie werkelijke kosten geboekt in combinatie met de verantwoordelijke werkplek 8400/ABTB. De zelfde contract+positie zonder verantwoordelijke werkplek 8400/ABTB geeft aan welke verwachtingscijfers er nog zijn voor de rest van het jaar.<br>

Aanvullend zie je ook dat de verwachtingscijfers op een contract+positie voor opdrachtgever TenneT weer leiden tot 0 op het niveau van de contract+positie in combinatie met de werkelijke kosten. Dit komt omdat hier geen afspraak over is vastgelegd. In de afspraak met Liander is hier wel een bedrag over afgesproken, zodat je echte aanvullende verwachtingscijfers ziet op contract+positie in combinatie met verantwoordelijke werkplek #/# voor opdrachtgever Liander.
    
2021: Voor SA-informatie is er geen EJV meer te verdelen over P, M en D. Daarom heeft extrapolatie obv van EJV ook geen zin meer.
</div>

In [ ]:
# Welke boekingsregels bestaan er reeds voor de contract+posities met verantwoordelijke werkplek 8400/ABTB in huidig boekjaar?
print("Voor de contractposities met verantwoordelijke werkplek 8400/ABTB in het huidige boekjaar betreft het onderstaande boekingsregels:")
display(df_sa[(df_sa["Verantw. werkplek"] == "8400/ABTB") & (df_sa["Boekjaar"] == str(huidig_boekjaar))])
if df_sa[(df_sa["Verantw. werkplek"] == "8400/ABTB")].groupby(["TV_correctie"]).sum().reset_index().shape[0] == 1:
    print("\nAlle contractposities met verantwoordelijke werkplek 8400/ATBT zijn voorzien van een correctiemelding:")
    display(df_sa[(df_sa["Verantw. werkplek"] == "8400/ABTB") & (df_sa["Boekjaar"] == str(huidig_boekjaar))].groupby(["RB: Opdrachtgever", "TV_correctie"]).sum().reset_index())

In [ ]:
print("Het totaal van EJV in het huidige boekjaar is {0:,.2f}".format(df_sa_regio.loc[df_sa_regio['Boekjaar'] == str(huidig_boekjaar)]['Cumulatieve werkelijkekosten'].sum()))

<a id="Indexeren"></a>
## Indexeren en eventueel lineair extrapoleren

<div class="alert alert-block alert-info">
Waardevelden in de boekjaren 2017 t/m 2019 die een bedrag vertegenwoordigen gaan we terugindexeren waarbij we 2020 zien als 100%. Verder gaan we op verzoek waardevelden in boekjaar 2020 extrapoleren om op bedragen uit te komen voor een heel jaar (zie ook variabele <code>aantal_maanden_meegenomen_in_2020</code> in combinatie met variable <code>extrapolatiemethode</code>).<br><br>

Voorbeeld:<br>
* Deltapercentage tussen 2019 en 2020 is +2,58%, dan zou een waarde van 100.00 in 2019 naar 102.58 teruggeindexeerd
 moeten worden.<br>
* Deltapercentage tussen 2018 en 2019 is +1,75%, dan zou een waarde van 100.00 in 2018 eerst naar 102.58 teruggeindexeerd moeten worden (indexatie naar 2019) om vervolgens nog naar 104,38 teruggeindexeerd moeten worden.<br>
    
<s><b>N.B.</b> Over de materiaalkosten en dienstenkosten hebben we geen "formele" indexatiecijfers. Daarom indexeren we alle bedragen met de zelfde factoren, ook al zijn deze factoren berekend op basis van de personeelskosten.</s><br>

Met de komst van de cijfers van BC hebben we formele indexatiecijfers!
</div>

**Voorbereiding indexeren en extrapoleren**

In [ ]:
# Variabelen toevoegen om teruggeindexeerde waarden en extrapolatie op te kunnen slaan
df_sa_regio["IND_werkelijke_kosten"] = 0
df_sa_regio["IND_personeelskosten"] = 0
df_sa_regio["IND_materiaalkosten"] = 0
df_sa_regio["IND_dienstenkosten"] = 0
df_sa_regio["IND_intercompanykosten"] = 0
df_sa_regio["IND_werkelijke_uren"] = 0
df_sa_regio["IND_Aantalgereedgemeld"] = 0

In [ ]:
# Voorbereiden op indexering
# In onderstaande indexering worden de cijfers overgenomen zoals BC die heeft verstrekt. Niet de cijfers die we zelf
# als deltapercentage hebben bepaald.
#indexering_dict={}
#indexering_dict["2017"]=1*((100+delta_percentages[0])/100)*((100+delta_percentages[1])/100)*((100+delta_percentages[2])/100)
#indexering_dict["2018"]=1*((100+delta_percentages[1])/100)*((100+delta_percentages[2])/100)
#indexering_dict["2019"]=1*((100+delta_percentages[2])/100)
#indexering_dict["2020"]=(1/aantal_maanden_meegenomen_in_2020)*12    # Extrapoleren
#print("De indexeringscijfers over de verschillende jaren zijn dus als volgt:")
#display(indexering_dict)

**Cijfers van BC**

Prijspeil CPI       | CPI Personeel | CPI Materialen | CPI Diensten
---                 |---            |---             |---
Prijspeil 2017>2020 | 1,119392505   | 1,040502       | 1,11576375
Prijspeil 2018>2020 | 1,071189      | 1,0302         | 1,08855
Prijspeil 2019>2020 | 1,029         | 1,020          | 1,062


Marcel,

De CPI gegevens voor 2021 zijn als volgt:
•	CPI Personeel            2%
•	CPI Materialen           1%
•	CPI Diensten              5%

Bron: Targetbrief 2021_2025 Financial Control-Karel Frederiks

Groet,
Theo


In [ ]:
# BC-cijfers opbouwen
indexering_dict={}
indexering_dict["2017"]=(1.119392505, 1.040502, 1.11576375)
indexering_dict["2018"]=(1.071189, 1.0302, 1.08855)
indexering_dict["2019"]=(1.029, 1.020, 1.062)

# Indexeringscijfers voor 2020:
indexering_dict["2020"]=(1.02, 1.01, 1.05)
indexering_dict["2021"]=(1, 1, 1)  # 2021 heeft nog geen indexering, dus allemaal * 1
laatste_indexering = indexering_dict["2020"]

for bj in ["2017", "2018", "2019"]:
    indexering_dict[bj]=( indexering_dict[bj][0] * laatste_indexering[0],
                          indexering_dict[bj][1] * laatste_indexering[1],
                          indexering_dict[bj][2] * laatste_indexering[2])

# #Vervolgbewerking afhankelijk van extrapolatiemethode
# if extrapolatiemethode == 1:
#     # Ja, we willen extrapoleren op basis van lineariteit
#     indexering_dict[str(huidig_boekjaar)]=((1/aantal_maanden_meegenomen_in_huidig_boekjaar)*12, 
#                                            (1/aantal_maanden_meegenomen_in_huidig_boekjaar)*12, 
#                                            (1/aantal_maanden_meegenomen_in_huidig_boekjaar)*12)    # Extrapoleren
# else:
#     indexering_dict["2020"]=(1, 1, 1)    # Niet extrapoleren, hou de getallen gelijk

# print("De indexeringscijfers over de verschillende jaren zijn dus als volgt:")
display(indexering_dict)

**Indexering en extrapolatie uitvoeren**

In [ ]:
# Maak een functie voor de indexering en extrapolatie
# Dit is de oude indexeringsfunctie, deze wordt niet uitgevoerd.
# def indexering_en_extrapolatie(row):
#     """Deze functie verzorgt de omrekening op basis van de factoren per boekjaar. De originele waardevelden worden
#        niet aangepast, alleen de extra toegevoegde velden (die beginnen allemaal met 'IND_').
#        Velden met aantallen worden niet met een factor aangepast, behalve de extrapolatiefactor voor 2020."""
#     # Vervolgbewerking afhankelijk of het boekjaar in de dictionary met factoren voor komt
#     if row["Boekjaar"] in indexering_dict:
#         # Regel bevat een boekjaar wat in de dictionary voorkomt, voer de berekeningen uit
#         factor = indexering_dict[row["Boekjaar"]]
#         row["IND_werkelijke_kosten"] = row['Cumulatieve werkelijkekosten'] * factor
#         row["IND_personeelskosten"] = row['Personeelskostencumulatief'] * factor
#         row["IND_materiaalkosten"] = row['Materiaalkostencumulatief'] * factor
#         row["IND_dienstenkosten"] = row['Dienstenkostencumulatief'] * factor
#         row["IND_intercompanykosten"] = row['Intercompanykostencumulatief'] * factor
#         # Vervolgbewerking afhankelijk van het boekjaar
#         if row["Boekjaar"] == '2020':
#             # Boekjaar = 2020, we passen de extrapolatiefactor toe op de aantallen
#             row["IND_werkelijke_uren"] = row['Werkelijke urenTotaal'] * factor                # Alleen extrapolatie
#             row["IND_Aantalgereedgemeld"] = row['SA: Aantalgereedgemeld(totaal)'] * factor    # Alleen extrapolatie
#         else:
#             # Boekjaar <> 2020, we nemen de aantallen over zonder factor
#             row["IND_werkelijke_uren"] = row['Werkelijke urenTotaal']                         # Geen factor over aantallen
#             row["IND_Aantalgereedgemeld"] = row['SA: Aantalgereedgemeld(totaal)']             # Geen factor over aantallen
#     else: # if row["Boekjaar"] not in indexering_dict
#         # Regel bevat geen boekjaar of een boekjaar wat niet in de dictionary zit. Dit mag niet voorkomen
#         print("UITVAL:",row)
#     return row

In [ ]:
# Deze wordt niet uitgevoerd
# Voer bovenstaande functie uit op het dataframe
#df_sa_regio = df_sa_regio.apply(func=indexering_en_extrapolatie, axis=1)
#print("Indexering en extrapolatie is uitgevoerd!")

In [ ]:
def indexering_en_extrapolatie_BC(row):
    """Deze functie verzorgt de omrekening op basis van de factoren per boekjaar. De originele waardevelden worden
       niet aangepast, alleen de extra toegevoegde velden (die beginnen allemaal met 'IND_').
       Velden met aantallen worden niet met een factor aangepast, behalve de extrapolatiefactor voor 2020."""
    # Vervolgbewerking afhankelijk of het boekjaar in de dictionary met factoren voor komt
    if row["Boekjaar"] in indexering_dict:
        # Regel bevat een boekjaar wat in de dictionary voorkomt, voer de berekeningen uit
        factor = indexering_dict[row["Boekjaar"]]
        #row["IND_werkelijke_kosten"] = row['Cumulatieve werkelijkekosten'] * factor[0]
        row["IND_personeelskosten"] = row['Personeelskostencumulatief'] * factor[0]
        row["IND_materiaalkosten"] = row['Materiaalkostencumulatief'] * factor[1]
        row["IND_dienstenkosten"] = row['Dienstenkostencumulatief'] * factor[2]
        row["IND_intercompanykosten"] = row['Intercompanykostencumulatief'] * factor[2]
        row["IND_werkelijke_kosten"] = ( row["IND_personeelskosten"] + row["IND_materiaalkosten"] + 
                                         row["IND_dienstenkosten"] + row["IND_intercompanykosten"] )
#         print(( row["IND_personeelskosten"] , row["IND_materiaalkosten"] , 
#                                          row["IND_dienstenkosten"] , row["IND_intercompanykosten"], "=",
#               ( float(row["IND_personeelskosten"]) + float(row["IND_materiaalkosten"]) + 
#                                          float(row["IND_dienstenkosten"]) + float(row["IND_intercompanykosten"]) ) ))
        # Vervolgbewerking afhankelijk van het boekjaar
        if row["Boekjaar"] == '2020':
            # Boekjaar = 2020, we passen de extrapolatiefactor ook toe op de aantallen
            row["IND_werkelijke_uren"] = row['Werkelijke urenTotaal'] * factor[0]                # Alleen extrapolatie
            row["IND_Aantalgereedgemeld"] = row['SA: Aantalgereedgemeld(totaal)'] * factor[0]    # Alleen extrapolatie
        else:
            # Boekjaar <> 2020, we nemen de aantallen over zonder factor
            row["IND_werkelijke_uren"] = row['Werkelijke urenTotaal']                         # Geen factor over aantallen
            row["IND_Aantalgereedgemeld"] = row['SA: Aantalgereedgemeld(totaal)']             # Geen factor over aantallen
    else: # if row["Boekjaar"] not in indexering_dict
        # Regel bevat geen boekjaar of een boekjaar wat niet in de dictionary zit. Dit mag niet voorkomen
        print("UITVAL:",row)
    return row

In [ ]:
# Voer bovenstaande functie uit op het dataframe
# Zorg eerst voor 0-waarden
df_sa_regio.fillna(value={"Cumulatieve werkelijkekosten": 0, 
                          "Personeelskostencumulatief": 0, 
                          "Materiaalkostencumulatief": 0, 
                          "Dienstenkostencumulatief": 0, 
                          "Intercompanykostencumulatief": 0, 
                          "Werkelijke urenTotaal": 0, 
                          "SA: Aantalgereedgemeld(totaal)":0},inplace=True)
# Voer nu de functie uit
df_sa_regio = df_sa_regio.apply(func=indexering_en_extrapolatie_BC, axis=1)
print("Indexering en extrapolatie is uitgevoerd!")

In [ ]:
# Oorspronkelijke waarden vergelijken met de nieuwe waarden
print("Werkelijke kosten in het oorspronkelijk dataframe:")
display(df_sa.pivot_table(index="RB: Opdrachtgever", columns="Boekjaar", values="Cumulatieve werkelijkekosten",
                          aggfunc="sum", margins=True))

print("De indexeringscijfers over de verschillende jaren zijn:")
display(indexering_dict)

# Vervolgbewerking afhankelijk van extrapolatiemethode
if extrapolatiemethode == 2:
    #Extrapolatiemethode is obv verwachtingscijfers
    print("Extrapolatie op basis van verdeling van de verwachtingscijfers naar rato.")
    print("Hierdoor is bij de werkelijke kosten de resterende EJV-verwachtingscijfers toegevoegd.")
    print("Het totaal van de cumulatieve werkelijkekosten (incl resterende EJV) in het huidige boekjaar is {0:,.2f}\n".format(df_sa_regio.loc[df_sa_regio['Boekjaar']==huidig_boekjaar]['Cumulatieve werkelijkekosten'].sum()))

print("Werkelijke kosten en geïndexeerde/geëxtrapoleerde kosten in het nieuwe dataframe:")
display(df_sa_regio.pivot_table(index="RB: Opdrachtgever", columns="Boekjaar", values=["Cumulatieve werkelijkekosten",
                          "IND_werkelijke_kosten"], aggfunc="sum", margins=True))

**Cijfers per jaar**

In [ ]:
# Op welke velden willen we opbossen
groupby_kolommen = ['Boekjaar', 'TV_correctie', 'RB: Opdrachtgever', 'RB: Business unit', 'TV_regio',
       'Productblad', 'Productbladomschrijving','Functie', 'TV_werk']

# Welke waardevelden nemen we mee
groupby_waardevelden = ['Cumulatieve werkelijkekosten', 'Personeelskostencumulatief', 'Materiaalkostencumulatief',
                        'Dienstenkostencumulatief', 'Intercompanykostencumulatief', 'Werkelijke urenTotaal',
                        'SA: Aantalgereedgemeld(totaal)',
                        "IND_werkelijke_kosten", "IND_personeelskosten", "IND_materiaalkosten",
                        "IND_dienstenkosten", "IND_intercompanykosten", "IND_werkelijke_uren",
                        "IND_Aantalgereedgemeld"]

print("Benodigde velden zijn gedefinieerd voor verdere opbossing.")

In [ ]:
# We laten het nieuwe dataframe onbewerkt door er een kopie van te maken
df_sa_regio_bewerkt = df_sa_regio.copy()

# Voor een juiste cumulatie in de groupby is het nodig om in de groupby-kolommen de niet-aanwezige waarden om te zetten naar
# een lege waarde
for i in groupby_kolommen:
    df_sa_regio_bewerkt[i] = df_sa_regio_bewerkt[i].replace(np.nan, '', regex=True)
print("Omvang van het nieuwe dataframe:",df_sa_regio.shape)
print("Omvang van het kopie dataframe met lege waarden ipv niet-aanwezige waarden:", df_sa_regio_bewerkt.shape)

# Om verder te gaan met het Dashboard in SAP BusinessObjects maken we het dataframe zo compact mogelijk.
# Grote dataframes maken SAP BusinessObjects erg traag.
df_sa_compact = df_sa_regio_bewerkt.groupby(groupby_kolommen)[groupby_waardevelden].sum().reset_index()
print("Omvang van het compacte dataframe:",df_sa_compact.shape)

In [ ]:
# Oorspronkelijke waarden vergelijken met de waarden in het compacte dataframe
print("Werkelijke kosten in het nieuwe dataframe:")
display(df_sa_regio.pivot_table(index="RB: Opdrachtgever", columns="Boekjaar", values=["Cumulatieve werkelijkekosten",
                          "IND_werkelijke_kosten"], aggfunc="sum", margins=True))

print("\nWerkelijke kosten en geïndexeerde/geëxtrapoleerde kosten in het compacte dataframe:")
display(df_sa_compact.pivot_table(index="RB: Opdrachtgever", columns="Boekjaar", values=["Cumulatieve werkelijkekosten",
                          "IND_werkelijke_kosten"], aggfunc="sum", margins=True))

**Productbladomschrijvingen recht trekken**

In november 2020 merkten we dat er een productblad was waarvan de productbladomschrijving was aangepast. Dit resulteert bij verder gebruik en/of concatenatie van productbladnummer met de productbladomschrijving dat je meerdere resultaten kreeg voor één en dezelfde key. Daarom wordt de omschrijving van het productblad gelijk getrokken over alle jaren heen met onderstaande code.

In [ ]:
# We doen onderstaande alleen voor de df_sa_compact-dataframe, omdat die verderop gebruikt wordt in de exports die naar de
# rapportagetools gaan

# Maak een dataframe van alleen de productbladnummers en hun omschrijvingen
print("Omvang van df_sa_compact:", df_sa_compact.shape)
df_productbladomschrijving = df_sa_compact[["Productblad","Productbladomschrijving"]]
print("Omvang van df_productbladomschrijving:", df_productbladomschrijving.shape)

# Weergave van enkele regels
display(df_productbladomschrijving.head())

# Ontdubbel waarbij de laatste (meest recente) omschrijving wordt bewaard (keep = "last")
print("Nu ontdubbelen!")
df_productbladomschrijving = df_productbladomschrijving.drop_duplicates(subset=['Productblad'], keep='last', inplace=False, ignore_index=True).copy()
print("Omvang van df_productbladomschrijving:", df_productbladomschrijving.shape)
display(df_productbladomschrijving.head())

# Productbladomschrijvingen van dataframe naar een dictionary voor eenvoudiger zoeken
dict_productbladomschrijving = df_productbladomschrijving.set_index('Productblad').to_dict("dict")
dict_productbladomschrijving = dict_productbladomschrijving["Productbladomschrijving"]
print("Aantal productbladregels in dictionary:",len(dict_productbladomschrijving))

print("Voorbeeld uit dictionary voor Productblad 142104:",dict_productbladomschrijving["142104"])

# Verwerken omschrijving in dataframe df_sa_compact
# Vervang eerst de productbladomschrijving door het productbladnummer
df_sa_compact["Productbladomschrijving"] = df_sa_compact["Productblad"]
# Gebruik daarna de dictionary om het productbladnummer te vervangen door de nieuwe omschrijving
df_sa_compact["Productbladomschrijving"].replace(to_replace=dict_productbladomschrijving, inplace=True)
display(df_sa_compact.head())
print("Omvang van df_sa_compact:", df_sa_compact.shape)

print("De productbladomschrijvingen zijn nu rechtgetrokken")

<a id="Excel"></a>
## Export naar Excel

In [ ]:
# Exporteren van de bestanden naar Excel
df_sa.to_excel("Originele samenvoeging van de Excelbestanden (2017-2020).xlsx") # M.u.v. anonimisering medewerkeromschrijving
df_sa_regio.to_excel("Detaillering met opsplitsing naar regio en indexering-extrapolatie.xlsx")
df_sa_compact.to_excel("PWC_SA.xlsx", index_label="Index") # Voor snelle verwerking in SAP BusinessObjects
print("Bestanden zijn succesvol geëxporteerd")

<div class="alert alert-block alert-success">
<b>Beschrijving van de kolommen van het "PWC_SA.xlsx"-bestand:</b><br></div>

* <b>Index</b>: Automatische nummering door pandas.
* <b>Boekjaar</b>: Boekjaar van de boekingsdocumenten/vastleggingen in de administratie. Voor 2017 t/m 2020 is dit initieel uit SAP BW geëxporteerd. Enige aanpassing is omzetting naar 4 cijfers. Daarna is dit hard toegevoegd bij het inlezen van de HANA-informatie (2021 en verder).  
* <b>TV_correctie</b>: Toegevoegd veld met de reden waarom een correctie uitgesloten wordt.  
* <b>RB: Opdrachtgever</b>: Naam van de opdrachtgever, zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>RB: Business unit</b>: Code van de businessunit, zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>TV_regio</b>: Toegevoegd veld met de regio of landelijke unit, afgeleid van de verantwoordelijke werkplek uit SAP BW (2017-2020) of HANA (2021-).
* <b>Productblad</b>: Productbladnummer van een gestandaard type werk, zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>Productbladomschrijving</b>: Productbladomschrijving behorende bij het productbladnummer en geeft het gestandaardiseerde type werk aan, zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>Functie</b>: Meest recente omschrijving functie van de medewerker zoals uit SAP geëxporteerd. Deze omschrijving heeft geen aanvullende letter zoals A, B, C of D, waarmee eventuele zwaartes worden aangegeven.
* <b>TV_werk</b>: Toegevoegd veld waarmee we het werk classificeren als 'Uitvoering' of 'Voorbereiding of nazorg', gebaseerd o.a. op routinggroep van de SA-order en functie van de medewerker.
* <b>Cumulatieve werkelijkekosten</b>: Alle werkelijke kosten zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd. Hierin zitten dus de hieronder genoemde personeelskosten, materiaalkosten, dienstenkosten en intercompanykosten.
* <b>Personeelskostencumulatief</b>: Alle personeelskosten zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>Materiaalkostencumulatief</b>: Alle materiaalkosten zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>Dienstenkostencumulatief</b>: Alle dienstenkosten zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>Intercompanykostencumulatief</b>: Alle intercompanykosten zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>Werkelijke urenTotaal</b>: Het aantal geboekte werkelijke uren zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>SA: Aantalgereedgemeld(totaal)</b>: Het aantal gereedgemelde SA-orders zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>IND_werkelijke_kosten</b>: Geïndexeerde werkelijke kosten (2017-vorige boekjaar) of niet-geïndexeerde werkelijke kosten (huidig boekjaar)
* <b>IND_personeelskosten</b>: Geïndexeerde personeelskosten (2017-vorige boekjaar) of niet-geïndexeerde personeelskosten (huidig boekjaar)
* <b>IND_materiaalkosten</b>: Geïndexeerde materiaalkosten (2017-vorige boekjaar) of niet-geïndexeerde materiaalkosten (huidig boekjaar)
* <b>IND_dienstenkosten</b>: Geïndexeerde dienstenkosten (2017-vorige boekjaar) of niet-geïndexeerde dienstenkosten (huidig boekjaar)
* <b>IND_intercompanykosten</b>: Geïndexeerde intercompanykosten (2017-vorige boekjaar) of niet-geïndexeerde intercompanykosten (huidig boekjaar)
* <b>IND_werkelijke_uren</b>: Werkelijke uren uit SAP BW (2017-2020) of HANA (2021-)
* <b>IND_Aantalgereedgemeld</b>: Aantal gereedgemeld uit SAP BW (2017-2020) of HANA (2021-)

In [ ]:
# Aan het eind het Excelbestand en PDF van Jupyter-notebook delen met Martijn Geerding (PWC), 
# cc Rianne ten Cate (PWC) (rianne.ten.cate@pwc.com), cc Bastiaan de Bruijn (Qirion), cc Theo Boomsma (Qirion)

Wil je meer weten over Jupyter-notebooks?
* In één minuut een introductie van Jupyter-notebooks: https://www.youtube.com/watch?v=eJDxcR1V7Qg (What are Jupyter Notebooks).
 [![Img AltText](http://img.youtube.com/vi/eJDxcR1V7Qg/0.jpg)](https://youtu.be/eJDxcR1V7Qg "What are Jupyter Notebooks")
* In onderstaande gratis training vind je in module 2 een tweetal video's over de Jupyter-notebook. Het bekijken hiervan kost nog geen 10 minuten: https://cognitiveclass.ai/courses/data-science-hands-open-source-tools-2

<a id="Ana1"></a>
## Analyse 1: Werkelijke kosten en aantal gereedgemeld verdeeld over de boekjaren

In [ ]:
# Kopie maken voor verdere bewerking of analyse
df = df_sa_compact.copy()

# Productbladcode en Productbladomschrijving in één veld
df["Productblad_met_omschrijving"] = df["Productblad"]+" - "+df["Productbladomschrijving"]

df.info()

In [ ]:
# Lege productbladen niet meenemen
print("Onderstaande waarde van lege productbladen wordt niet meegenomen:")
display(df[df["Productblad"]==""].pivot_table(index=["Productblad_met_omschrijving", "TV_correctie"], columns="Boekjaar", 
                          values=["Cumulatieve werkelijkekosten","SA: Aantalgereedgemeld(totaal)"], 
                          aggfunc="sum", margins=True))

**Analyse van productbladen op basis van cumulatieve werkelijkekosten per productblad per boekjaar**

In [ ]:
# Cumulatieve werkelijkekosten van niet-lege productbladen wordt wel meegenomen
df_pb = df[df["Productblad"]!=""].pivot_table(index="Productblad_met_omschrijving", columns="Boekjaar", 
                          values="Cumulatieve werkelijkekosten",
                          aggfunc="sum")
print("Er zijn",df_pb.shape[0], "productbladen met een totale Cumulatieve werkelijkekosten van: {0:,.2f}".format(df_pb["2017"].sum()+df_pb["2018"].sum()+df_pb["2019"].sum()+df_pb["2020"].sum()))

# Vervang 0-waarden door waarde 'niet-bekend (nan)'.
df_pb.replace(to_replace=0, value=np.nan, inplace=True)
print("\nIndien een productblad in een boekjaar een 0-waarde heeft als Cumulatieve werkelijkekosten,\nwordt deze als 'niet-bekend' meegenomen in onderstaande telling.")
print("Onderstaand het aantal niet-bekende productbladen per boekjaar (van uit die",df_pb.shape[0],"productbladen):")
df_pb.isna().sum().to_frame(name="Aantal niet-bekende productbladen")

In [ ]:
# Maak een functie om de volledigheid te controleren
def check_volledigheid(row):
    """Controleer voor welke jaren er informatie aanwezig is per productblad.
       Veld 'Volledigheid' dient vooraf aanwezig te zijn.
       De letter 'X' geeft aan of er voor dat jaar informatie aanwezig is.
       Een minteken '-' geeft aan dat er voor dat jaar geen informatie aanwezig is (of dat de waarde gelijk is aan 0)."""
    x = "XXXX"
    for i,yyyy in enumerate(['2017', '2018', '2019', '2020']):
        if str(row[yyyy])==str(np.nan):
            x = x[:i] + "-" + x[i + 1:]
#        elif float(row[yyyy])==float(0.0):        # Controle op 0-waarden toegevoegd dd 27-aug-2020
#            x = x[:i] + "-" + x[i + 1:]
    row["Volledigheid"] = x
    return row
print("Functie geactiveerd")

In [ ]:
# Definieer extra veld
df_pb["Volledigheid"] = ""
# Voer nu de functie uit
df_pb = df_pb.apply(func=check_volledigheid, axis=1)
print("Volledigheidscontrole is uitgevoerd!")

In [ ]:
# Voorbeeld van de kolom Volledigheid
df_pb.head()

In [ ]:
# Extra modules importeren voor grafische weergave
import seaborn as sns
import matplotlib.pyplot as plt

# Haal de draaitabel uit elkaar met de jaren 2017 tm 2020 in de kolom Boekjaar en de waarden als Cumulatieve werkelijkekosten
df_pb_melt =pd.melt(df_pb.reset_index()#.fillna(value=0) 
                    ,id_vars=['Productblad_met_omschrijving', 'Volledigheid'], 
                    value_vars=['2017', '2018', '2019', '2020'],
                    var_name="Boekjaar", value_name="Cumulatieve werkelijkekosten")

# Aggregeer de aantallen op volledigheid en geef ze weer over de boekjaren
piv1 = pd.pivot_table(df_pb_melt, index=["Volledigheid"], 
                            columns="Boekjaar", 
                            values="Cumulatieve werkelijkekosten",
                            aggfunc="count",
                            fill_value=0)
# Aggregeer de aantallen op volledigheid
piv2 = pd.pivot_table(df_pb.reset_index(), index=["Volledigheid"], 
                            #columns=['2017', '2018', '2019', '2020'], 
                            #columns="Boekjaar", 
                            values="Productblad_met_omschrijving",
                            aggfunc="count",
                            fill_value=0)

print("In de kolom volledigheid kan je zien welke jaren gevuld zijn.\n",
      "De vier X-karakters staan achtereenvolgens voor 2017, 2018, 2019 en 2020.")
print("---X staat voor 2020 alleen. X-XX staat voor 2017, niets, 2019 en 2020.")

f, ax = plt.subplots(figsize=(15, 6))
sns.heatmap(piv1, annot=True,fmt="d",linewidths=.5, cmap="Blues",ax=ax)
plt.setp(ax.yaxis.get_majorticklabels(), rotation=0, fontname="Courier New", fontsize=20)
ax.title.set_text("Aantal productbladen per volledigheid op basis van aanwezigheid 'Cumulatieve werkelijkekosten'")
plt.show()
print("NB: Er zijn",df_pb[df_pb["Volledigheid"]=="----"].shape[0], "productbladen die niet gevuld zijn in de aanwezige boekjaren (hierboven niet zichtbaar in de rij '----').")

In [ ]:
# Aggregeer de waarde op volledigheid
piv3 = df_pb.reset_index().fillna(value=0)
piv3["Totaal"] = piv3["2017"]+piv3["2018"]+piv3["2019"]+piv3["2020"]
piv3 = pd.pivot_table(piv3, index=["Volledigheid"], 
                            #columns=['2017', '2018', '2019', '2020'], 
                            #columns="Boekjaar", 
                            values="Totaal",
                            aggfunc="sum",
                            fill_value=0)

print("Onderstaande verdeling is op basis van de aanwezigheid van cumulatieve werkelijkekosten (ongelijk 0) in een boekjaar:")
# Geef zowel de aantallen als de gerelateerde cumulatieve kosten weer per volledigheid
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2,figsize=(15, 6))
sns.heatmap(piv2, annot=True,fmt="d",linewidths=.5, cmap="Blues",ax=ax1)
sns.heatmap(piv3, annot=True,fmt="1.1f",linewidths=.5, cmap="Blues",ax=ax2)
plt.setp(ax1.yaxis.get_majorticklabels(), rotation=0, fontname="Courier New", fontsize=20)
plt.setp(ax2.yaxis.get_majorticklabels(), rotation=0, fontname="Courier New", fontsize=20)
ax1.title.set_text('Aantal productbladen per volledigheid op basis van kosten')
plt.setp(ax1, xlabel=str(piv2["Productblad_met_omschrijving"].sum())+" productbladen")
plt.setp(ax2, xlabel="{0:,.2f}".format(piv3["Totaal"].sum())+" EUR")
ax2.title.set_text('Kosten per volledigheid op basis van kosten')
for t in ax2.texts:
    t.set_text('{0:,.2f}'.format(float(t.get_text())))
plt.show()

In [ ]:
piv2a = piv2.reset_index()[(piv2.reset_index()["Volledigheid"].str.startswith("X")) & (piv2.reset_index()["Volledigheid"].str.endswith("X"))] #[["Volledigheid","Productblad_met_omschrijving"]]
piv2a.columns.name = "index"
print("Aantal productbladen aanwezig in 2017 en in 2020:",
      piv2a["Productblad_met_omschrijving"].sum(), 
      "van totaal aantal productbladen",
      piv2["Productblad_met_omschrijving"].sum(), 
      "( = {0:.1f}% )".format((float(piv2a["Productblad_met_omschrijving"].sum())/float(piv2["Productblad_met_omschrijving"].sum()))*100))
display(piv2a)

In [ ]:
piv3a = piv3.reset_index()[(piv2.reset_index()["Volledigheid"].str.startswith("X")) & (piv2.reset_index()["Volledigheid"].str.endswith("X"))] #[["Volledigheid","Productblad_met_omschrijving"]]
piv3a.columns.name = "index"
print("Werkelijke kosten van productbladen aanwezig in 2017 en in 2020: {0:,.2f}".format(piv3a["Totaal"].sum()), 
      "van totaal werkelijke kosten van alle productbladen {0:,.2f}".format(piv3["Totaal"].sum()),
      "( = {0:.1f}% )".format((float(piv3a["Totaal"].sum())/float(piv3["Totaal"].sum()))*100))
display(piv3a)

**Analyse van productbladen op basis van aantal gereedgemeld per productblad per boekjaar**

In [ ]:
# Aantal gereedgemeld van niet-lege productbladen wordt wel meegenomen
df_pba = df[df["Productblad"]!=""].pivot_table(index="Productblad_met_omschrijving", columns="Boekjaar", 
                          values="SA: Aantalgereedgemeld(totaal)",
                          aggfunc="sum")
print("Er zijn",df_pba.shape[0], "productbladen met een totale gereedgemelde aantallen van:", df_pba["2017"].sum()+df_pba["2018"].sum()+df_pba["2019"].sum()+df_pba["2020"].sum())

# Vervang 0-waarden door waarde 'niet-bekend (nan)'.
df_pba.replace(to_replace=0, value=np.nan, inplace=True)
print("\nIndien een productblad in een boekjaar een 0-waarde als aantal gereedgemeld heeft, wordt deze als 'niet-bekend' meegenomen in onderstaande telling.")

print("Onderstaand het aantal niet-bekende productbladen per boekjaar (van uit die",df_pba.shape[0],"productbladen):")
df_pba.isna().sum().to_frame(name="Aantal niet-bekende productbladen")

In [ ]:
# Definieer extra veld
df_pba["Volledigheid"] = ""
# Voer nu de functie uit
df_pba = df_pba.apply(func=check_volledigheid, axis=1)
print("Volledigheidscontrole is uitgevoerd!")

In [ ]:
# Haal de draaitabel uit elkaar met de jaren 2017 tm 2020 in de kolom Boekjaar en de waarden als Cumulatieve werkelijkekosten
df_pba_melt =pd.melt(df_pba.reset_index(), id_vars=['Productblad_met_omschrijving', 'Volledigheid'], 
                    value_vars=['2017', '2018', '2019', '2020'],
                    var_name="Boekjaar", value_name="Gereedgemelde aantallen")

# Aggregeer de aantallen op volledigheid en geef ze weer over de boekjaren
piva1 = pd.pivot_table(df_pba_melt, index=["Volledigheid"], 
                            columns="Boekjaar", 
                            values="Gereedgemelde aantallen",
                            aggfunc="count",
                            fill_value=0)
# Aggregeer de aantallen op volledigheid
piva2 = pd.pivot_table(df_pba.reset_index(), index=["Volledigheid"], 
                            #columns=['2017', '2018', '2019', '2020'], 
                            #columns="Boekjaar", 
                            values="Productblad_met_omschrijving",
                            aggfunc="count",
                            fill_value=0)

print("In de kolom volledigheid kan je zien welke jaren gevuld zijn. De vier X-karakters staan achtereenvolgens voor 2017, 2018, 2019 en 2020.")
print("---X staat voor 2020 alleen. X-XX staat voor 2017, niets, 2019 en 2020.")

f, ax = plt.subplots(figsize=(15, 6))
sns.heatmap(piva1, annot=True,fmt="d",linewidths=.5, cmap="Blues",ax=ax)
plt.setp(ax.yaxis.get_majorticklabels(), rotation=0, fontname="Courier New", fontsize=20)
ax.title.set_text("Aantal productbladen per volledigheid op basis van aanwezigheid 'aantal gereedgemeld'")
plt.show()
print("NB: Er zijn",df_pba[df_pba["Volledigheid"]=="----"].shape[0], "productbladen die niet gevuld zijn in de aanwezige boekjaren (hierboven niet zichtbaar in de rij '----').")

In [ ]:
df_pba = df_pba.merge(df_pb, how='left', left_index=True, right_index=True, suffixes=('_aantal', '_waarde'), copy=True, indicator=False, validate=None)


In [ ]:
# Aggregeer de waarde op volledigheid
piva3 = df_pba.reset_index().fillna(value=0)
piva3["Totaal"] = piva3["2017_waarde"]+piva3["2018_waarde"]+piva3["2019_waarde"]+piva3["2020_waarde"]
piva3 = pd.pivot_table(piva3, index=["Volledigheid_aantal"], 
                            #columns=['2017', '2018', '2019', '2020'], 
                            #columns="Boekjaar", 
                            values="Totaal",
                            aggfunc="sum",
                            fill_value=0)

# Geef zowel de aantallen als de gerelateerde cumulatieve kosten weer per volledigheid
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2,figsize=(15, 6))
sns.heatmap(piva2, annot=True,fmt="d",linewidths=.5, cmap="Blues",ax=ax1)
sns.heatmap(piva3, annot=True,fmt="1.1f",linewidths=.5, cmap="Blues",ax=ax2)
plt.setp(ax1.yaxis.get_majorticklabels(), rotation=0, fontname="Courier New", fontsize=20)
plt.setp(ax2.yaxis.get_majorticklabels(), rotation=0, fontname="Courier New", fontsize=20)
ax1.title.set_text('Aantal productbladen per volledigheid op basis van aantal gereedgemeld')
plt.setp(ax1, xlabel=str(piva2["Productblad_met_omschrijving"].sum())+" productbladen")
plt.setp(ax2, xlabel="{0:,.2f}".format(piva3["Totaal"].sum())+" EUR")
ax2.title.set_text('Kosten per volledigheid op basis van aantal gereedgemeld')
for t in ax2.texts:
    t.set_text('{0:,.2f}'.format(float(t.get_text())))
plt.show()

In [ ]:
piva2a = piva2.reset_index()[(piva2.reset_index()["Volledigheid"].str.startswith("X")) & (piva2.reset_index()["Volledigheid"].str.endswith("X"))] #[["Volledigheid","Productblad_met_omschrijving"]]
piva2a.columns.name = "index"
print("Aantal productbladen aanwezig in 2017 en in 2020:",
      piva2a["Productblad_met_omschrijving"].sum(), 
      "van totaal aantal productbladen",
      piva2["Productblad_met_omschrijving"].sum(), 
      "( = {0:.1f}% )".format((float(piva2a["Productblad_met_omschrijving"].sum())/float(piva2["Productblad_met_omschrijving"].sum()))*100))
display(piva2a)

In [ ]:
piva3a = piva3.reset_index()[(piva2.reset_index()["Volledigheid"].str.startswith("X")) & (piva2.reset_index()["Volledigheid"].str.endswith("X"))] #[["Volledigheid","Productblad_met_omschrijving"]]
piva3a.columns.name = "index"
print("Werkelijke kosten van productbladen aanwezig in 2017 en in 2020: {0:,.2f}".format(piva3a["Totaal"].sum()), 
      "van totaal werkelijke kosten van alle productbladen {0:,.2f}".format(piva3["Totaal"].sum()),
      "( = {0:.1f}% )".format((float(piva3a["Totaal"].sum())/float(piva3["Totaal"].sum()))*100))
display(piva3a)

**Welke productbladen hebben wel kosten, maar geen aantallen?**

In [ ]:
# Vul de nan-velden in de waarde-kolommen. Dit is nodig om een totaalkolom toe te voegen
df_pba.fillna(value={'2017_waarde':0, '2018_waarde':0, '2019_waarde':0, '2020_waarde':0}, inplace=True)
# Voeg totaalkolom toe
df_pba["Totaalwaarde"]=df_pba['2017_waarde'] + df_pba['2018_waarde'] + df_pba['2019_waarde'] + df_pba['2020_waarde']
# Beschrijf samenvatting hiervan
print("Er zijn", df_pba.loc[df_pba["Volledigheid_aantal"]=="----"].shape[0],"productbladen zonder aantallen.",
      "Hierop staat een totaalwaarde van {0:,.2f}".format(df_pba["Totaalwaarde"].loc[df_pba["Volledigheid_aantal"]=="----"].sum()),"EUR op.")

In [ ]:
# Geef overzicht van deze productbladen met hun totaalwaarde
df_pba[['2017_aantal', '2018_aantal', '2019_aantal', '2020_aantal',
       'Volledigheid_aantal', 'Volledigheid_waarde', 'Totaalwaarde']].loc[df_pba["Volledigheid_aantal"]=="----"]

**Welke productbladen hebben wel kosten in alle jaren, maar geen aantallen in alle jaren?**

In [ ]:
# Beschrijf samenvatting hiervan
print("Er zijn", df_pba.loc[(df_pba["Volledigheid_aantal"]!="XXXX") & (df_pba["Volledigheid_waarde"]=="XXXX")].shape[0],"productbladen zonder aantallen in alle jaren.",
      "Hierop staat een totaalwaarde van {0:,.2f}".format(df_pba["Totaalwaarde"].loc[(df_pba["Volledigheid_aantal"]!="XXXX") & (df_pba["Volledigheid_waarde"]=="XXXX")].sum()),"EUR op.")

In [ ]:
df_pba[['2017_aantal', '2018_aantal', '2019_aantal', '2020_aantal',
       'Volledigheid_aantal', 'Volledigheid_waarde', 'Totaalwaarde']].loc[(df_pba["Volledigheid_aantal"]!="XXXX") & (df_pba["Volledigheid_waarde"]=="XXXX")]

In [ ]:
# Maak een "Volledigheid_totaal"-kolom en plaats daarin de volledigheid van de waarde en de volledigheid van de aantallen
df_pba["Volledigheid_totaal"]=df_pba['Volledigheid_waarde'] + " : " + df_pba['Volledigheid_aantal']

In [ ]:
# Aggregeer de waarde op volledigheid
pivaw3 = df_pba.reset_index().fillna(value=0)
pivaw3["Totaal"] = pivaw3["2017_waarde"]+pivaw3["2018_waarde"]+pivaw3["2019_waarde"]+pivaw3["2020_waarde"]
pivaw3 = pd.pivot_table(pivaw3, index=["Volledigheid_totaal"], 
                            #columns=['2017', '2018', '2019', '2020'], 
                            #columns="Boekjaar", 
                            values="Totaal",
                            aggfunc="sum",
                            fill_value=0)

# Aggregeer de aantallen op volledigheid
pivaw2 = pd.pivot_table(df_pba.reset_index(), index=["Volledigheid_totaal"], 
                            #columns=['2017', '2018', '2019', '2020'], 
                            #columns="Boekjaar", 
                            values="Productblad_met_omschrijving",
                            aggfunc="count",
                            fill_value=0)

print("In de kolom Volledigheid_totaal zie je 2 groepjes van 4 karakters.")
print("In het eerste groepje kan je zien in welke jaren een waarde gevuld is.\nDe vier X-karakters staan achtereenvolgens voor 2017, 2018, 2019 en 2020.")
print("Het tweede groepje (achter het :-teken) laat zien in welke jaren het aantal gereedgemeld gevuld is.")
print("---X staat voor 2020 alleen. X-XX staat voor 2017, niets, 2019 en 2020.")


# Geef zowel de aantallen als de gerelateerde cumulatieve kosten weer per volledigheid
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2,figsize=(15, 15))
#fig, ax2 = plt.subplots(nrows=1, ncols=1,figsize=(15, 15))
sns.heatmap(pivaw2, annot=True,fmt="d",linewidths=.5, cmap="Blues",ax=ax1)
sns.heatmap(pivaw3, annot=True,fmt="1.1f",linewidths=.5, cmap="Blues",ax=ax2)
plt.setp(ax1.yaxis.get_majorticklabels(), rotation=0, fontname="Courier New", fontsize=20)
plt.setp(ax2.yaxis.get_majorticklabels(), rotation=0, fontname="Courier New", fontsize=20)
ax1.title.set_text('Aantal productbladen per volledigheid op waarde : aantal')
plt.setp(ax1, xlabel=str(pivaw2["Productblad_met_omschrijving"].sum())+" productbladen")
plt.setp(ax2, xlabel="{0:,.2f}".format(pivaw3["Totaal"].sum())+" EUR")
ax2.title.set_text('Kosten per volledigheid op waarde : aantal')
for t in ax2.texts:
    t.set_text('{0:,.2f}'.format(float(t.get_text())))
plt.tight_layout()
plt.show()

<a id="Ana2"></a>
## Analyse 2: Controle voor dashboards

In [ ]:
# Kopie maken voor verdere bewerking of analyse
df_db = df_sa_compact.copy()

# Productbladcode en Productbladomschrijving in één veld is handiger in sommige overzichten
df_db["Productblad_met_omschrijving"] = df_db["Productblad"]+" - "+df_db["Productbladomschrijving"]

df_db.info()

In [ ]:
# Vervolgbewerking afhankelijk van extrapolatiemethode
if extrapolatiemethode == 1:
    # Extrapolatie voorbereiden
    print("Kosten van het jaar 2020 dienen nog geëxtrapoleerd te worden, vandaar onderstaande controlegetallen.")

    # Controle vooraf van 2 (willekeurige) waarden in twee verschillende boekjaren waaronder 2020
    print("Cumulatieve werkelijke kosten uit 2018, deze moet gelijk blijven: {0:,.2f}".format(df_db["Cumulatieve werkelijkekosten"].loc[df_db["Boekjaar"] == "2018"].sum()))
    print("Personeelskosten uit 2018, deze moet gelijk blijven: {0:,.2f}".format(df_db["Personeelskostencumulatief"].loc[df_db["Boekjaar"] == "2018"].sum()))
    print("Cumulatieve werkelijke kosten uit 2020, deze moet veranderen: {0:,.2f}".format(df_db["Cumulatieve werkelijkekosten"].loc[df_db["Boekjaar"] == "2020"].sum()))
    print("Personeelskosten uit 2020, deze moet veranderen: {0:,.2f}".format(df_db["Personeelskostencumulatief"].loc[df_db["Boekjaar"] == "2020"].sum()))
    
    # Alle niet geïndexeerde waarden van 2020 toch extrapoleren. Dit maakt het makkelijk voor het dashboard
    df_db.loc[df_db["Boekjaar"] == "2020", ["Cumulatieve werkelijkekosten", "Personeelskostencumulatief", 
                                            "Materiaalkostencumulatief", "Dienstenkostencumulatief", 
                                            "Intercompanykostencumulatief", "Werkelijke urenTotaal",
                                            "SA: Aantalgereedgemeld(totaal)"]] *= ((1/aantal_maanden_meegenomen_in_2020)*12)

    # Controle achteraf van 2 (willekeurige) waarden in twee verschillende boekjaren waaronder 2020
    print("\nControle achteraf, na het extrapoleren:")
    print("Cumulatieve werkelijke kosten uit 2018, deze moet gelijk blijven: {0:,.2f}".format(df_db["Cumulatieve werkelijkekosten"].loc[df_db["Boekjaar"] == "2018"].sum()))
    print("Personeelskosten uit 2018, deze moet gelijk blijven: {0:,.2f}".format(df_db["Personeelskostencumulatief"].loc[df_db["Boekjaar"] == "2018"].sum()))
    print("Cumulatieve werkelijke kosten uit 2020, deze moet veranderen: {0:,.2f}".format(df_db["Cumulatieve werkelijkekosten"].loc[df_db["Boekjaar"] == "2020"].sum()))
    print("Personeelskosten uit 2020, deze moet veranderen: {0:,.2f}".format(df_db["Personeelskostencumulatief"].loc[df_db["Boekjaar"] == "2020"].sum()))
else:
    print("Geen extrapolatie op dit punt in de analyse")

In [ ]:
# Extra kolommen toevoegen voor eenvoudigere berekeningen
extra_kolommen_list = ["TV_Pers_uitvoering", "TV_Pers_overig", "TV_IND_Pers_uitvoering", "TV_IND_Pers_overig"]
for i in extra_kolommen_list:
    df_db[i] = 0.0
df_db.info()

In [ ]:
# Weergave van eerste 10 regels voordat personeelskosten gesplitst worden
display(df_db.head(10))

# Splitsen van personeelskosten
df_db.loc[df_db["TV_werk"] == "Uitvoering", "TV_Pers_uitvoering"] = df_db["Personeelskostencumulatief"]
df_db.loc[df_db["TV_werk"] != "Uitvoering", "TV_Pers_overig"] = df_db["Personeelskostencumulatief"]
df_db.loc[df_db["TV_werk"] == "Uitvoering", "TV_IND_Pers_uitvoering"] = df_db["IND_personeelskosten"]
df_db.loc[df_db["TV_werk"] != "Uitvoering", "TV_IND_Pers_overig"] = df_db["IND_personeelskosten"]

# Weergave van eerste 10 regels nadat personeelskosten gesplitst zijn
display(df_db.head(10))

<div class="alert alert-block alert-success">
<b>**Export naar Excel met extra velden**</b><br></div>

In [ ]:
# Speciaal op verzoek van Ben een extra export naar Excel
# Dit is nu de BRON voor de Dashboards in BOBI en SAC
df_db.to_excel("Productiviteit_SA_extra_velden.xlsx", index_label="Index") # Voor snelle verwerking in SAP BusinessObjects
df_db_totaal = df_db.copy()                                     # Extra kopie met de juiste informatie

<div class="alert alert-block alert-success">
<b>Beschrijving van de kolommen van het "Productiviteit_SA_extra_velden.xlsx"-bestand:</b><br></div>

* <b>Index</b>: Automatische nummering door pandas.
* <b>Boekjaar</b>: Boekjaar van de boekingsdocumenten/vastleggingen in de administratie. Voor 2017 t/m 2020 is dit initieel uit SAP BW geëxporteerd. Enige aanpassing is omzetting naar 4 cijfers. Daarna is dit hard toegevoegd bij het inlezen van de HANA-informatie (2021 en verder).  
* <b>TV_correctie</b>: Toegevoegd veld met de reden waarom een correctie uitgesloten wordt.  
* <b>RB: Opdrachtgever</b>: Naam van de opdrachtgever, zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>RB: Business unit</b>: Code van de businessunit, zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>TV_regio</b>: Toegevoegd veld met de regio of landelijke unit, afgeleid van de verantwoordelijke werkplek uit SAP BW (2017-2020) of HANA (2021-).
* <b>Productblad</b>: Productbladnummer van een gestandaard type werk, zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>Productbladomschrijving</b>: Productbladomschrijving behorende bij het productbladnummer en geeft het gestandaardiseerde type werk aan, zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>Functie</b>: Meest recente omschrijving functie van de medewerker zoals uit SAP geëxporteerd. Deze omschrijving heeft geen aanvullende letter zoals A, B, C of D, waarmee eventuele zwaartes worden aangegeven.
* <b>TV_werk</b>: Toegevoegd veld waarmee we het werk classificeren als 'Uitvoering' of 'Voorbereiding of nazorg', gebaseerd o.a. op routinggroep van de SA-order en functie van de medewerker.
* <b>Cumulatieve werkelijkekosten</b>: Alle werkelijke kosten zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd. Hierin zitten dus de hieronder genoemde personeelskosten, materiaalkosten, dienstenkosten en intercompanykosten.
* <b>Personeelskostencumulatief</b>: Alle personeelskosten zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>Materiaalkostencumulatief</b>: Alle materiaalkosten zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>Dienstenkostencumulatief</b>: Alle dienstenkosten zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>Intercompanykostencumulatief</b>: Alle intercompanykosten zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>Werkelijke urenTotaal</b>: Het aantal geboekte werkelijke uren zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>SA: Aantalgereedgemeld(totaal)</b>: Het aantal gereedgemelde SA-orders zoals uit SAP BW (2017-2020) of HANA (2021-) geëxporteerd.
* <b>IND_werkelijke_kosten</b>: Geïndexeerde werkelijke kosten (2017-vorige boekjaar) of niet-geïndexeerde werkelijke kosten (huidig boekjaar)
* <b>IND_personeelskosten</b>: Geïndexeerde personeelskosten (2017-vorige boekjaar) of niet-geïndexeerde personeelskosten (huidig boekjaar)
* <b>IND_materiaalkosten</b>: Geïndexeerde materiaalkosten (2017-vorige boekjaar) of niet-geïndexeerde materiaalkosten (huidig boekjaar)
* <b>IND_dienstenkosten</b>: Geïndexeerde dienstenkosten (2017-vorige boekjaar) of niet-geïndexeerde dienstenkosten (huidig boekjaar)
* <b>IND_intercompanykosten</b>: Geïndexeerde intercompanykosten (2017-vorige boekjaar) of niet-geïndexeerde intercompanykosten (huidig boekjaar)
* <b>IND_werkelijke_uren</b>: Werkelijke uren uit SAP BW (2017-2020) of HANA (2021-)
* <b>IND_Aantalgereedgemeld</b>: Aantal gereedgemeld uit SAP BW (2017-2020) of HANA (2021-)
* <b>Productblad_met_omschrijving</b>: Bovengenoemd Productblad en Productbladomschrijving met behulp van een min-teken met elkaar verboven in één veld ter gebruik in omschrijvingen en selecties
* <b>TV_Pers_uitvoering</b>: Werkelijke personeelskosten (gelijk aan het veld Personeelskostencumulatief), echter alleen toegekend indien het veld TV_werk gelijk is aan 'Uitvoering'. Dit maakt het gebruik in een vervolgapplicatie eenvoudiger
* <b>TV_Pers_overig</b>: Werkelijke personeelskosten (gelijk aan het veld Personeelskostencumulatief), echter alleen toegekend indien het veld TV_werk gelijk is aan 'Voorbereiding of nazorg'. Dit maakt het gebruik in een vervolgapplicatie eenvoudiger
* <b>TV_IND_Pers_uitvoering</b>: Geïndexeerde personeelskosten (2017-vorige boekjaar) of niet-geïndexeerde personeelskosten (huidig boekjaar) (gelijk aan het veld IND_personeelskosten), echter alleen toegekend indien het veld TV_werk gelijk is aan 'Uitvoering'. Dit maakt het gebruik in een vervolgapplicatie eenvoudiger
* <b>TV_IND_Pers_overig</b>: Geïndexeerde personeelskosten (2017-vorige boekjaar) of niet-geïndexeerde personeelskosten (huidig boekjaar) (gelijk aan het veld IND_personeelskosten), echter alleen toegekend indien het veld TV_werk gelijk is aan 'Voorbereiding of nazorg'. Dit maakt het gebruik in een vervolgapplicatie eenvoudiger

In [ ]:
# Verwijderen rijen met correctie-informatie
print("Shape van dataframe voor verwijderen correctie-informatie:",df_db.shape)
df_db = df_db[df_db["TV_correctie"]==""].copy()
print("Shape van dataframe na verwijderen correctie-informatie:",df_db.shape)
# Extra kopie t.b.v. filteringen op productblad of regio
df_db_kopie = df_db.copy()             # Let op: In deze kopie zijn de correcties reeds afwezig!

In [ ]:
print("Lijst van",df_db["Productblad"].nunique(),"productbladnummers:\n",sorted(list(df_db["Productblad"].unique())))
print("Lijst van",df_db["TV_regio"].nunique(),"regio's:\n",sorted(list(df_db["TV_regio"].unique())))

<div class="alert alert-block alert-success">
<b>**Filtering voor Productblad en/of regio**</b><br></div>

In [ ]:
# Vul hier eventueel productbladnummer en/of regio in
Selectie_Productblad = ""
Selectie_Regio = ""

In [ ]:
# Haal de extra kopie op
df_db               = df_db_kopie.copy()
# Voer dit ook uit voor de versie waarin de correcties nog aanwezig zijn
df_db_met_correctie = df_db_totaal.copy()

# Kijk of de selectievelden gevuld zijn en indien ja, selecteer hier dan ook op!
if Selectie_Productblad != "":
    # Selectie_Productblad is gevuld
    print("Shape van dataframe voor selectie op productblad",Selectie_Productblad,":",df_db.shape, df_db_met_correctie.shape)
    df_db = df_db[df_db["Productblad"] == Selectie_Productblad].copy()
    df_db_met_correctie = df_db_met_correctie[df_db_met_correctie["Productblad"] == Selectie_Productblad].copy()
    print("Shape van dataframe na selectie op productblad",Selectie_Productblad,":",df_db.shape, df_db_met_correctie.shape)
if Selectie_Regio != "":
    print("Shape van dataframe voor selectie op regio", Selectie_Regio, ":",df_db.shape, df_db_met_correctie.shape)
    df_db = df_db[df_db["TV_regio"] == Selectie_Regio].copy()
    df_db_met_correctie = df_db_met_correctie[df_db_met_correctie["TV_regio"] == Selectie_Regio].copy()
    print("Shape van dataframe na selectie op regio", Selectie_Regio, ":",df_db.shape, df_db_met_correctie.shape)

print("Aantal productbladnummers:\n", df_db["Productblad"].nunique(), df_db_met_correctie["Productblad"].nunique())
print("Aantal regio's:\n", df_db["TV_regio"].nunique(), df_db_met_correctie["TV_regio"].nunique())
print("Lijst van regio's:\n", df_db["TV_regio"].unique())

In [ ]:
df_db_pb = df_db.groupby(by=['Productblad','Productblad_met_omschrijving','Boekjaar'],as_index=False).sum()
display(df_db_pb.head())
print(df_db_pb.shape)

In [ ]:
# Bepaal populatie aan de hand van het feit of er aantal gereedgemeld voorkomen voor elk jaar
# Indien ja, productblad blijft in populatie
# Indien nee, productblad verdwijnt uit populatie
df_db_pb_populatie = df_db_pb.pivot(index="Productblad", columns="Boekjaar", values="IND_Aantalgereedgemeld").reset_index()
df_db_pb_populatie.columns.name=""
df_db_pb_populatie

In [ ]:
# Verander alle nan-waarden in 0 (nan="not a number")
df_db_pb_populatie.fillna(value=0, inplace=True)
df_db_pb_populatie

In [ ]:
# Maak een lijst van alle productbladen waarvan elk jaar een aantal gereed heeft wat ongelijk 0 is.
pb_list = df_db_pb_populatie.loc[(df_db_pb_populatie!=0).all(axis=1)]["Productblad"].to_list()
print("Er zijn nog",len(pb_list), "productbladen over, waaronder: ", list(pb_list)[:5],"...")

In [ ]:
# Filter het restant aan productbladen waar we mee door willen gaan
df_db_pb_restant = df_db_pb.loc[(df_db_pb["Productblad"].isin(pb_list))].copy()
print("Shape van het nog niet op het productblad gefilterde dataframe:",df_db_pb.shape)
print("Shape van het wel op het productblad gefilterde dataframe:",df_db_pb_restant.shape)
print("Aantal productbladen in de lijst * 4:",len(pb_list)*4)

In [ ]:
# Indruk van het restant aan productbladen
df_db_pb_restant

In [ ]:
# Bereken de gemiddelde P, M en D-kosten
df_db_pb_restant["Gem_P_kosten_uitvoering"] = df_db_pb_restant["TV_Pers_uitvoering"] / df_db_pb_restant["SA: Aantalgereedgemeld(totaal)"]
df_db_pb_restant["Gem_P_kosten_voorbereiding_nazorg"] = df_db_pb_restant["TV_Pers_overig"] / df_db_pb_restant["SA: Aantalgereedgemeld(totaal)"]
df_db_pb_restant["Gem_M_kosten"] = df_db_pb_restant["Materiaalkostencumulatief"] / df_db_pb_restant["SA: Aantalgereedgemeld(totaal)"]
df_db_pb_restant["Gem_D_kosten"] = df_db_pb_restant["Dienstenkostencumulatief"] / df_db_pb_restant["SA: Aantalgereedgemeld(totaal)"]

df_db_pb_restant["Gem_IND_P_kosten_uitvoering"] = df_db_pb_restant["TV_IND_Pers_uitvoering"] / df_db_pb_restant["IND_Aantalgereedgemeld"]
df_db_pb_restant["Gem_IND_P_kosten_voorbereiding_nazorg"] = df_db_pb_restant["TV_IND_Pers_overig"] / df_db_pb_restant["IND_Aantalgereedgemeld"]
df_db_pb_restant["Gem_IND_M_kosten"] = df_db_pb_restant["IND_materiaalkosten"] / df_db_pb_restant["IND_Aantalgereedgemeld"]
df_db_pb_restant["Gem_IND_D_kosten"] = df_db_pb_restant["IND_dienstenkosten"] / df_db_pb_restant["IND_Aantalgereedgemeld"]


In [ ]:
# Nu is het dataframe voorzien van de gemiddelde P, M en D-kosten (laatste kolommen)
df_db_pb_restant

In [ ]:
# Haal het hoogste boekjaar op
hoogste_boekjaar = df_db_pb_restant["Boekjaar"].max()
print("Het hoogste boekjaar is:",hoogste_boekjaar, "met als variabletype",(type(hoogste_boekjaar)))

In [ ]:
# Filter alleen op de regels met het hoogste boekjaar
df_hoogste_boekjaar = df_db_pb_restant.loc[(df_db_pb_restant["Boekjaar"]==hoogste_boekjaar)]
display(df_hoogste_boekjaar.head())
print("Shape van dit dataframe:", df_hoogste_boekjaar.shape)

In [ ]:
# Neem nu alleen de hoogst nodige kolommen
df_hoogste_boekjaar = df_hoogste_boekjaar[['Productblad', 'TV_Pers_uitvoering', 'TV_Pers_overig',
                                           'Materiaalkostencumulatief', 'Dienstenkostencumulatief',
                                           'SA: Aantalgereedgemeld(totaal)', 'TV_IND_Pers_uitvoering', 
                                           'TV_IND_Pers_overig', 'IND_materiaalkosten', 'IND_dienstenkosten',
                                           'IND_Aantalgereedgemeld', 'Gem_P_kosten_uitvoering', 
                                           'Gem_P_kosten_voorbereiding_nazorg', 'Gem_M_kosten', 'Gem_D_kosten',
                                           'Gem_IND_P_kosten_uitvoering', 'Gem_IND_P_kosten_voorbereiding_nazorg',
                                           'Gem_IND_M_kosten', 'Gem_IND_D_kosten']]
# Pas de omschrijving van deze kolommen aan: voeg de letters HB toe (Hoogste Boekjaar) 
df_hoogste_boekjaar.columns = [x+"_HB" for x in df_hoogste_boekjaar.columns]
display(df_hoogste_boekjaar.head())

In [ ]:
# Vul de waardevelden van het hoogste boekjaar toe aan het dataframe waarin ook andere boekjaren staan (JOIN)
df_db_pb_restant = df_db_pb_restant.merge(df_hoogste_boekjaar, how='inner', left_on="Productblad", right_on="Productblad_HB")
display(df_db_pb_restant.loc[(df_db_pb_restant["Productblad"]=="143011")])

In [ ]:
# Berekeningen
df_db_pb_restant["GK_P_uitvoerend"]  = ( df_db_pb_restant['TV_IND_Pers_uitvoering_HB'] - 
                                         ( df_db_pb_restant['Gem_IND_P_kosten_uitvoering_HB'] - df_db_pb_restant['Gem_IND_P_kosten_uitvoering']) 
                                         * df_db_pb_restant['IND_Aantalgereedgemeld_HB'] )
df_db_pb_restant["GK_P_voor_nazorg"] = ( df_db_pb_restant['TV_IND_Pers_overig_HB'] - 
                                         ( df_db_pb_restant['Gem_IND_P_kosten_voorbereiding_nazorg_HB'] - df_db_pb_restant['Gem_IND_P_kosten_voorbereiding_nazorg']) 
                                         * df_db_pb_restant['IND_Aantalgereedgemeld_HB'] )
df_db_pb_restant["GK_Materiaal"]     = ( df_db_pb_restant['IND_materiaalkosten_HB'] - 
                                         ( df_db_pb_restant['Gem_IND_M_kosten_HB'] - df_db_pb_restant['Gem_IND_M_kosten'])
                                         * df_db_pb_restant['IND_Aantalgereedgemeld_HB'] )
df_db_pb_restant["GK_Diensten"]      = ( df_db_pb_restant['IND_dienstenkosten_HB'] - 
                                         ( df_db_pb_restant['Gem_IND_D_kosten_HB'] - df_db_pb_restant['Gem_IND_D_kosten'])
                                         * df_db_pb_restant['IND_Aantalgereedgemeld_HB'] )
    
df_db_pb_restant["Kvc_P_uitvoerend"]  = ( df_db_pb_restant['TV_Pers_uitvoering_HB'] - 
                                          ( df_db_pb_restant['Gem_P_kosten_uitvoering_HB'] - df_db_pb_restant['Gem_P_kosten_uitvoering']) 
                                          * df_db_pb_restant['IND_Aantalgereedgemeld_HB'] )
df_db_pb_restant["Kvc_P_voor_nazorg"] = ( df_db_pb_restant['TV_Pers_overig_HB'] - 
                                          ( df_db_pb_restant['Gem_P_kosten_voorbereiding_nazorg_HB'] - df_db_pb_restant['Gem_P_kosten_voorbereiding_nazorg']) 
                                          * df_db_pb_restant['IND_Aantalgereedgemeld_HB'] )
df_db_pb_restant["Kvc_Materiaal"]     = ( df_db_pb_restant['Materiaalkostencumulatief_HB'] - 
                                          ( df_db_pb_restant['Gem_M_kosten_HB'] - df_db_pb_restant['Gem_M_kosten'])
                                          * df_db_pb_restant['IND_Aantalgereedgemeld_HB'] )
df_db_pb_restant["Kvc_Diensten"]      = ( df_db_pb_restant['Dienstenkostencumulatief_HB'] - 
                                          ( df_db_pb_restant['Gem_D_kosten_HB'] - df_db_pb_restant['Gem_D_kosten'])
                                          * df_db_pb_restant['IND_Aantalgereedgemeld_HB'] )
        
df_db_pb_restant["C_P_uitvoerend"]    = df_db_pb_restant["GK_P_uitvoerend"]  - df_db_pb_restant["Kvc_P_uitvoerend"]
df_db_pb_restant["C_P_voor_nazorg"]   = df_db_pb_restant["GK_P_voor_nazorg"] - df_db_pb_restant["Kvc_P_voor_nazorg"]
df_db_pb_restant["C_Materiaal"]       = df_db_pb_restant["GK_Materiaal"]     - df_db_pb_restant["Kvc_Materiaal"]
df_db_pb_restant["C_Diensten"]        = df_db_pb_restant["GK_Diensten"]      - df_db_pb_restant["Kvc_Diensten"]


In [ ]:
# Test om te vergelijken met een handmatig uitgewerkt overzicht voor productblad 143011
display(df_db_pb_restant.loc[(df_db_pb_restant["Productblad"]=="143011")])

In [ ]:
# Totalenoverzicht
print("Totalenoverzicht per boekjaar:")
df_db_pb_restant.groupby(by="Boekjaar")[['GK_P_uitvoerend', 'GK_P_voor_nazorg', 'GK_Materiaal', 'GK_Diensten',
                              'C_P_uitvoerend', 'C_P_voor_nazorg', 'C_Materiaal', 'C_Diensten',
                              'Kvc_P_uitvoerend', 'Kvc_P_voor_nazorg', 'Kvc_Materiaal', 'Kvc_Diensten']].sum().T
                       

In [ ]:
# Export van bovenstaande totalenoverzicht naar Excel
# test = df_db_pb_restant.groupby(by="Boekjaar")[['GC_P_uitvoerend', 'GK_P_voor_nazorg', 'GK_Materiaal', 'GK_Diensten',
#                               'C_P_uitvoerend', 'C_P_voor_nazorg', 'C_Materiaal', 'C_Diensten',
#                               'Kvc_P_uitvoerend', 'Kvc_P_voor_nazorg', 'Kvc_Materiaal', 'Kvc_Diensten']].sum().T
                       
# test.to_excel("PWC_SA_totalen.xlsx", index_label="Index")

In [ ]:
# Overzicht per productblad en boekjaar
df_db_pb_restant.groupby(by=["Productblad","Boekjaar"])[['GK_P_uitvoerend', 'GK_P_voor_nazorg', 'GK_Materiaal', 'GK_Diensten',
                              'C_P_uitvoerend', 'C_P_voor_nazorg', 'C_Materiaal', 'C_Diensten',
                              'Kvc_P_uitvoerend', 'Kvc_P_voor_nazorg', 'Kvc_Materiaal', 'Kvc_Diensten']].sum().T
                       

In [ ]:
# Export van bovenstaande totalenoverzicht per productblad naar Excel
# test = df_db_pb_restant.groupby(by=["Productblad","Boekjaar"])[['GC_P_uitvoerend', 'GK_P_voor_nazorg', 'GK_Materiaal', 'GK_Diensten',
#                               'C_P_uitvoerend', 'C_P_voor_nazorg', 'C_Materiaal', 'C_Diensten',
#                               'Kvc_P_uitvoerend', 'Kvc_P_voor_nazorg', 'Kvc_Materiaal', 'Kvc_Diensten']].sum().T

#test.to_excel("PWC_SA_totalen_per_productblad.xlsx", index_label="Index")

In [ ]:
# Totalenoverzicht
print("Totalenoverzicht van de gecorrigeerde kosten:")
display(df_db_pb_restant.groupby(by="Boekjaar")[['GK_P_uitvoerend', 'GK_P_voor_nazorg', 'GK_Materiaal', 'GK_Diensten'
                              ]].sum().T)
print("Met totalen:")
display(df_db_pb_restant.groupby(by="Boekjaar")[['GK_P_uitvoerend', 'GK_P_voor_nazorg', 'GK_Materiaal', 'GK_Diensten'
                              ]].sum().T.sum().to_frame(name="Totaal").T)

In [ ]:
print("Herkenbaarheid van de cijfers:")
# Opbouwen dataframe voor output
# Kosten
df_output = df_db_met_correctie.groupby(by="Boekjaar")[["Cumulatieve werkelijkekosten"]].sum().T.sum().to_frame(name="Totale kosten (EUR)").T
df_output = pd.concat([df_output,df_db_met_correctie.loc[df_db_met_correctie["TV_correctie"] != ""].groupby(by="Boekjaar")[["Cumulatieve werkelijkekosten"]].sum().T.sum().to_frame(name="Totale kosten niet toe te kennen aan een productblad (EUR)").T]) 
df_output = pd.concat([df_output,df_db_met_correctie.loc[df_db_met_correctie["TV_correctie"] == ""].groupby(by="Boekjaar")[["Cumulatieve werkelijkekosten"]].sum().T.sum().to_frame(name="Totale kosten wel toe te kennen aan een productblad (EUR)").T]) 

# lege regel
df_output = pd.concat([df_output,df_db_met_correctie.loc[df_db_met_correctie["TV_correctie"] == "AAA"].groupby(by="Boekjaar")[["Cumulatieve werkelijkekosten"]].sum().T.sum().to_frame(name=" ").T]) 

# Aantal gereedgemeld
df_output = pd.concat([df_output,df_db_met_correctie.loc[df_db_met_correctie["TV_correctie"] == ""].groupby(by="Boekjaar")[['SA: Aantalgereedgemeld(totaal)']].sum().T.sum().to_frame(name="Totale aantal productbladen gereedgemeld (aantal)").T]) 

# lege regel
df_output = pd.concat([df_output,df_db_met_correctie.loc[df_db_met_correctie["TV_correctie"] == "AAA"].groupby(by="Boekjaar")[["Cumulatieve werkelijkekosten"]].sum().T.sum().to_frame(name=" ").T]) 

# Kosten van voor elk jaar aanwezige productbladen (XXXX)
df_output = pd.concat([df_output,df_db_met_correctie.loc[(df_db_met_correctie["TV_correctie"] == "") & (df_db_met_correctie["Productblad"].isin(pb_list))].groupby(by="Boekjaar")[["Cumulatieve werkelijkekosten"]].sum().T.sum().to_frame(name="In scope kosten (Productbladen X-X-X-X) (EUR)").T]) 

# Aantal gereedgemeld van voor elk jaar aanwezige productbladen (XXXX)
df_output = pd.concat([df_output,df_db_met_correctie.loc[(df_db_met_correctie["TV_correctie"] == "") & (df_db_met_correctie["Productblad"].isin(pb_list))].groupby(by="Boekjaar")[['SA: Aantalgereedgemeld(totaal)']].sum().T.sum().to_frame(name="In scope aantal productbladen (Productbladen X-X-X-X) (aantal)").T]) 

# Assen verwisselen
df_output = df_output.T.copy()

# Percentages toekennen
df_output["% in scope kosten (%)"] = df_output['In scope kosten (Productbladen X-X-X-X) (EUR)'] / df_output['Totale kosten wel toe te kennen aan een productblad (EUR)'] * 100
df_output["% in scope aantal productbladen (%)"] = df_output['In scope aantal productbladen (Productbladen X-X-X-X) (aantal)'] / df_output['Totale aantal productbladen gereedgemeld (aantal)'] * 100

display(df_output.T.fillna(""))

## KPI Productiviteit SA (2021 en verder)

In [ ]:
# We laten het nieuwe dataframe onbewerkt door er een kopie van te maken
df_kpi = df_sa_regio.copy()

In [ ]:
# We doen onderstaande alleen voor de df_kpi-dataframe, omdat die verderop gebruikt wordt in de exports die naar de
# rapportagetools gaan

# Maak een dataframe van alleen de productbladnummers en hun omschrijvingen
print("Omvang van df_kpi:", df_kpi.shape)
df_productbladomschrijving = df_kpi[["Productblad","Productbladomschrijving"]]
print("Omvang van df_productbladomschrijving:", df_productbladomschrijving.shape)

# Weergave van enkele regels
display(df_productbladomschrijving.head())

# Ontdubbel waarbij de laatste (meest recente) omschrijving wordt bewaard (keep = "last")
print("Nu ontdubbelen!")
df_productbladomschrijving = df_productbladomschrijving.drop_duplicates(subset=['Productblad'], keep='last', inplace=False, ignore_index=True).copy()
print("Omvang van df_productbladomschrijving na ontdubbeling:", df_productbladomschrijving.shape)
display(df_productbladomschrijving.head())

# Productbladomschrijvingen van dataframe naar een dictionary voor eenvoudiger zoeken
dict_productbladomschrijving = df_productbladomschrijving.set_index('Productblad').to_dict("dict")
dict_productbladomschrijving = dict_productbladomschrijving["Productbladomschrijving"]
print("Aantal productbladregels in dictionary:",len(dict_productbladomschrijving))

print("Voorbeeld uit dictionary voor Productblad 142104:",dict_productbladomschrijving["142104"])

# Verwerken omschrijving in dataframe df_sa_compact
# Vervang eerst de productbladomschrijving door het productbladnummer
df_kpi["Productbladomschrijving"] = df_kpi["Productblad"]
# Gebruik daarna de dictionary om het productbladnummer te vervangen door de nieuwe omschrijving
df_kpi["Productbladomschrijving"].replace(to_replace=dict_productbladomschrijving, inplace=True)
display(df_kpi.head())
print("Omvang van df_kpi:", df_kpi.shape)

print("De productbladomschrijvingen zijn nu rechtgetrokken")

In [ ]:
# Op welke velden willen we opbossen
#groupby_kolommen = ['Boekjaar', 'TV_correctie', 'RB: Opdrachtgever', 'RB: Business unit', 'TV_regio',
#       'Productblad', 'Productbladomschrijving','Functie', 'TV_werk']

# Even zonder functie van de medewerker en direct/indirect (2021)
groupby_kolommen = ['Boekjaar', 'TV_correctie', 'RB: Opdrachtgever', 'RB: Business unit', 'TV_regio',
       'Productblad', 'Productbladomschrijving']

# Welke waardevelden nemen we mee
groupby_waardevelden = ['Cumulatieve werkelijkekosten', 'Personeelskostencumulatief', 'Materiaalkostencumulatief',
                        'Dienstenkostencumulatief', 'Intercompanykostencumulatief', 'Werkelijke urenTotaal',
                        'SA: Aantalgereedgemeld(totaal)',
                        "IND_werkelijke_kosten", "IND_personeelskosten", "IND_materiaalkosten",
                        "IND_dienstenkosten", "IND_intercompanykosten", "IND_werkelijke_uren",
                        "IND_Aantalgereedgemeld"]

print("Benodigde velden zijn gedefinieerd voor verdere opbossing.")

In [ ]:
# Voor een juiste cumulatie in de groupby is het nodig om in de groupby-kolommen de niet-aanwezige waarden om te zetten naar
# een lege waarde
for i in groupby_kolommen:
    df_kpi[i] = df_kpi[i].replace(np.nan, '', regex=True)
print("Omvang van het oorspronkelijke dataframe:",df_sa_regio.shape)
print("Omvang van het kopie dataframe met lege waarden ipv niet-aanwezige waarden:", df_kpi.shape)

# Om verder te gaan maken we het dataframe zo compact mogelijk.
df_kpi_compact = df_kpi.groupby(groupby_kolommen)[groupby_waardevelden].sum().reset_index()
print("Omvang van het compacte dataframe:",df_kpi_compact.shape)

In [ ]:
df_kpi_compact.to_excel("Productiviteit_SA_extra_velden_KPI2021.xlsx", index_label="Index") # Voor snelle verwerking in SAP BusinessObjects

In [ ]:
df_kpi.columns

In [ ]:
df_kpi_compact.columns

In [ ]:
# Aggregeren in twee stappen
# 1e stap: Correcties niet meenemen
df_kpi_compact_agg = df_kpi_compact[df_kpi_compact["TV_correctie"]==""].copy()

print("Zijn er nog correcties aanwezig?:", df_kpi_compact_agg["TV_correctie"].unique())
assert df_kpi_compact_agg["TV_correctie"].unique() == [""] , "Er zijn nog ongewenste correcties aanwezig"

# 2e stap: De echte aggregatie op boekjaar en productblad
groupby_kolommen = ['Boekjaar', 'Productblad']

# Welke waardevelden nemen we mee
groupby_waardevelden = ['Cumulatieve werkelijkekosten', 'Personeelskostencumulatief', 'Materiaalkostencumulatief',
                        'Dienstenkostencumulatief', 'Intercompanykostencumulatief', 'Werkelijke urenTotaal',
                        'SA: Aantalgereedgemeld(totaal)',
                        "IND_werkelijke_kosten", "IND_personeelskosten", "IND_materiaalkosten",
                        "IND_dienstenkosten", "IND_intercompanykosten", "IND_werkelijke_uren",
                        "IND_Aantalgereedgemeld"]

print("Benodigde velden zijn gedefinieerd voor verdere opbossing.")
# Voor een juiste cumulatie in de groupby is het nodig om in de groupby-kolommen de niet-aanwezige waarden om te zetten naar
# een lege waarde
for i in groupby_kolommen:
    df_kpi_compact_agg[i] = df_kpi_compact_agg[i].replace(np.nan, '', regex=True)
print("Omvang van het dataframe voor aggregatie:",df_kpi_compact_agg.shape)

# Om verder te gaan maken we het dataframe zo compact mogelijk.
df_kpi_compact_agg = df_kpi_compact_agg.groupby(groupby_kolommen)[groupby_waardevelden].sum().reset_index()
print("Omvang van het dataframe na aggregatie:",df_kpi_compact_agg.shape)

In [ ]:
df_kpi_compact_agg.head()

In [ ]:
# Extra kolommen toevoegen voor het vorige jaar
PY_kolommen = ['PY_IND_werkelijke_kosten', 'PY_IND_personeelskosten', 'PY_IND_materiaalkosten', 'PY_IND_dienstenkosten',
               'PY_IND_intercompanykosten', 'PY_IND_werkelijke_uren']
for kolom in PY_kolommen:
    df_kpi_compact[kolom]=np.nan

In [ ]:
def voeg_gemiddelde_vorig_jaar_toe(row):
    """Voegt aan het df_kpi_compact-dataframe de waarden van vorig jaar toe op basis van het gemiddelde van vorig jaar
       vermenigvuldigd met de aantallen van dit jaar"""
    temp = df_kpi_compact_agg[(df_kpi_compact_agg["Boekjaar"] == str(int(row["Boekjaar"])-1)) & 
                              (df_kpi_compact_agg["Productblad"] == row["Productblad"])].copy()
    # Vervolgbewerking afhankelijk van het vinden van het productblad in het vorig jaar
    if temp.shape[0] == 0:
        # Productblad in vorig jaar niet gevonden, neem 1:1 dit jaar over
        for kolom in PY_kolommen:
            row[kolom] = row[kolom[3:]]
            #print(row["Boekjaar"],row["Productblad"])
            #display(row.to_frame())
    else:
        # Productblad in vorig jaar wel gevonden!
        for kolom in PY_kolommen:
            if (temp.iloc[0]["IND_Aantalgereedgemeld"] != 0) and (row["IND_Aantalgereedgemeld"] != 0):
                row[kolom] = temp.iloc[0][kolom[3:]] / float(temp.iloc[0]["IND_Aantalgereedgemeld"]) * row["IND_Aantalgereedgemeld"]
            else:
                row[kolom] = row[kolom[3:]]
    return row

In [ ]:
# Filter op alle boekjaren muv de oudste
boekjaarfilter = sorted(df_kpi_compact_agg["Boekjaar"].unique())[1:]
print("Boekjaarfilter:", boekjaarfilter)
df_kpi_compact_temp = df_kpi_compact.loc[(df_kpi_compact["Boekjaar"].isin(boekjaarfilter)) & (df_kpi_compact["TV_correctie"] == "")].copy()
df_kpi_compact2 = df_kpi_compact_temp.apply(func=voeg_gemiddelde_vorig_jaar_toe, axis=1)                                    

In [ ]:
df_kpi_compact2.to_excel("KPI Productiviteit SA.xlsx")